# Livres hebdo rentrée litt 2023

![logo LH](https://pbs.twimg.com/profile_images/1315992576995065858/RzrhYGSZ_400x400.jpg)

### Objectifs
L'objectif ici est de réaliser une datavisualisation autour de la rentrée littéraire 2022 de façon amusante avec les infos fournies dans le dossier de Livres Hebdo. Le fichier mis à disposition étant au format pdf, j'ai dans un premier temps réaliser une extraction des données à l'aide des libriaires pdfreader et PyPDF2 (permet de lire et parser le contenu d'un pdf).
Dans un deuxième temps, le résultat obtenu n'étant pas suffisament satisfaisant, je suis passés par une etape de mise au propre et de structuration de la table dans Excel.
Troisième étape, finalisation de la structuration des données dans jupyter pour ensuite faire de l'enrichissement.

#### Idées :
- nb de titres recensés comme faisant partis de la R.L
- nb de titre par litt fr, litt étrangère, essais
- nb de titre par traduction (map des pays)
- nb de titre par genre (femme - homme)
- les époques où le récit à lieu
- les lieux où les récits ont lieu
- nb de 1er romans
- nb de titre avec préface
- nb de pages total (tous livres confondus)
- nb de pages moyen
- prix moyen
- budget total
- format moyen
- les mots les plus fréquents dans les résumés
- les mots les plus fréquents dans les titres

Liens :
- [Liste des ouvrages au format PDF](https://www.livreshebdo.fr/sites/default/files/2022-07/Bibliographie%20rentre%CC%81e%20litte%CC%81raire%202022.pdf)

##### Importation des librairies nécessaires pour le script

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import requests

import json
#from pandas.io.json import json_normalize
from pandas import json_normalize
import pandas as pd
import numpy as np
import datetime as dt  #pour l'ajout de la date de l'extraction de données

import streamlit as st

import matplotlib.pyplot as plt

import pickle

## Dico RL et PL

In [2]:
def load_data_dict(url):
    df_rl_dataviz_pl = pd.read_pickle(url) #"./dict_rl_final_23.pkl" pd.DataFrame.from_dict(pd.read_pickle(url)).T
    return df_rl_dataviz_pl

In [53]:
#je charge le dictionnaire 
dico_rl_dataviz = load_data_dict("dict_rl_final_20231126.pkl")

#je transforme le dictionnaire en dataframe
df_rl_dataviz_pl = pd.DataFrame.from_dict(dico_rl_dataviz, orient='index')

[(i,x['nom_complet'],x['livre']) for i,x in enumerate(dico_rl_dataviz.values()) if x['nom_complet'] == 'Lilia Hassaine']

[(377,
  'Lilia Hassaine',
  {'id': '',
   'titre': 'Panorama- Grand Format',
   'ean': 9782073035059.0,
   'maison_edition': 'Gallimard',
   'type': nan,
   'premier_roman': 'NAN',
   'date de parution': '2023-08-17',
   'mois de parution': 'August',
   'annee_parution': 2023.0,
   'traduit_de': nan,
   'traduit_par': nan,
   'resume': '',
   'caracteristiques': {'nombre_pages': '235 pages',
    'prix_indicatif': '20,00 €',
    'prix_ebook_indicatif': '14,99 €',
    'reliure': 'Broché',
    'format': 'Grand Format'},
   'autres_infos': {'couverture': 'https://products-images.di-static.com/image/lilia-hassaine-panorama/9782073035059-45x70-1.webp 1x,https://products-images.di-static.com/image/lilia-hassaine-panorama/9782073035059-45x70-2.webp 2x',
    'preface': '',
    'texte_autre': '',
    'illustration_photo_image': '',
    'oeuvre_original': ''},
   'prix_litteraire': {0.0: {'id_prix': '10RE',
     'nom_prix': 'RENAUDOT',
     'id_prix_detail': '10RE1RO',
     'nom_prix_detail': 'R

In [46]:
dico_rl_dataviz[139]['livre']['prix_litteraire'][1]['lauréat'] = 'OUI'

In [47]:
with open('dict_rl_final_20231126.pkl', 'wb') as fp:
		pickle.dump(dico_rl_dataviz, fp)

In [122]:
#pour tous les livres, sélectionner les auteurs 
test  = pd.DataFrame([[x['nom_complet'],x['livre']['titre'],x['livre']['maison_edition'],y] for x in list_livre_rl for y in x['livre']['prix_litteraire'].values() if y['nom_prix_detail'] == 'FEMINA_FRANCAIS'], columns=['Auteur','Livre','maison_edition','prix'])
#pd.json_normalize(test.Livre)
test = pd.concat([test, pd.json_normalize(test.pop("prix"))], axis=1)
test

,Auteur,Livre,maison_edition,id_prix,nom_prix,id_prix_detail,nom_prix_detail,annee,premiere_selection,deuxieme_selection,troisieme_selection,lauréat
0,Agnès Mathieu-Daudé,Marchands de sable- Grand Format,Flammarion,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,OUI,OUI,NON
1,Alice Renard,La Colère et l'Envie- Grand Format,Héloïse d'Ormesson,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,NON,NON,NON
2,Eric Chacour,Ce que je sais de toi- Grand Format,Philippe Rey,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,OUI,NON,NON
3,Lyonel Trouillot,Veilleuse du Calvaire- Grand Format,Actes Sud,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,NON,NON,NON
4,Lionel Duroy,Mes pas dans leurs ombres- Grand Format,Mialet Barrault,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,NON,NON,NON
5,Guy Boley,A ma soeur et unique- Grand Format,Grasset,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,OUI,OUI,NON
6,Benoît Coquil,Petites choses- Grand Format,Rivages,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,NON,NON,NON
7,Dominique Fabre,Gare Saint-Lazare- Grand Format,Fayard,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,NON,NON,NON
8,Sarah Chiche,Les alchimies- Grand Format,Seuil,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,OUI,NON,NON
9,Agnès Desarthe,Le château des rentiers- Grand Format,Editions de l'Olivier,2FE,FEMINA,2FE1FR,FEMINA_FRANCAIS,2023,OUI,OUI,NON,NON


In [115]:
[x['livre'] for x in list_livre_rl]

[{'id': '',
  'titre': 'Marchands de sable- Grand Format',
  'ean': 9782080411693.0,
  'maison_edition': 'Flammarion',
  'type': 'Romans français',
  'premier_roman': 'NON',
  'date de parution': '2023-08-30',
  'mois de parution': 'August',
  'annee_parution': 2023.0,
  'traduit_de': 'VF',
  'traduit_par': '-',
  'resume': '',
  'caracteristiques': {'nombre_pages': '336 pages',
   'prix_indicatif': '21,00 €',
   'prix_ebook_indicatif': '14,99 €',
   'reliure': 'Broché',
   'format': 'Grand Format'},
  'autres_infos': {'couverture': 'https://products-images.di-static.com/image/agnes-mathieu-daude-marchands-de-sable/9782080411693-45x70-1.webp 1x,https://products-images.di-static.com/image/agnes-mathieu-daude-marchands-de-sable/9782080411693-45x70-2.webp 2x',
   'preface': '',
   'texte_autre': '',
   'illustration_photo_image': '',
   'oeuvre_original': ''},
  'prix_litteraire': {0.0: {'id_prix': '2FE',
    'nom_prix': 'FEMINA',
    'id_prix_detail': '2FE1FR',
    'nom_prix_detail': 'FE

In [6]:
list_livre_rl = list(filter(lambda x: x['livre']['RL'] != '', dico_rl_dataviz.values()))
df_list_livre_rl = pd.DataFrame(list_livre_rl)
df_list_livre_rl

,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
0,,,,Agnès Mathieu-Daudé,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Marchands de sable- Grand..."
1,,,,Charlotte Milandri,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Au sol- Grand Format', 'e..."
2,,,,Jean-Michel Guenassia,M,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'A Dieu vat- Grand Format'..."
3,,,,Catherine Poulain,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'L'ombre d'un grand oiseau..."
4,,,,Charly Delwart,M,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Que ferais-je à ma place ..."
...,...,...,...,...,...,...,...,...,...,...
520,,,,Jean-Pierre Otte,M,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Mes anticorps- Grand Form..."
521,,,,Sabine Garrigues,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Rien n'est su- Grand Form..."
522,,,,Antoine Wauters,M,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Le plus court chemin- Gra..."
523,,,,Marielle Macé,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Respire- Poche', 'ean': 9..."


In [51]:
display(df_list_livre_rl.loc[df_list_livre_rl['nom_complet']== a])#.apply(lambda x: x['livre']['RL'] != '', dico_rl_dataviz.values()))


Agnès Desarthe


,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
56,,,,Agnès Desarthe,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Le château des rentiers- ..."


Agnès Mathieu-Daudé


,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
0,,,,Agnès Mathieu-Daudé,F,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Marchands de sable- Grand..."


Akira Mizubayashi


...

,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
147,,,,Valérie Mréjen,F,FRANCE,NaN,NaN,EUROPE,"{'id': '', 'titre': 'La jeune artiste- Poche',..."


Vincent Delecroix


,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
62,,,,Vincent Delecroix,M,FRANCE,46.603354,1.888334,EUROPE,"{'id': '', 'titre': 'Naufrage- Grand Format', ..."


Vincent Lefèvre


,id,nom,prenom,nom_complet,genre,pays,latitude,longitude,continent,livre
107,,,,Vincent Lefèvre,M,FRANCE,NaN,NaN,EUROPE,"{'id': '', 'titre': 'Le génie de l'art indien-..."


## Pour Dataviz

In [45]:
df_doublon = pd.read_csv("liste_rl_total_sans_doublon.csv", sep=";")#pd.read_pickle("liste_rl_total_sans_doublon.pkl")
display(df_doublon.head(), df_doublon.iloc[1289])

,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,Editeur,PAYS,latitude,...,prix_ebook,couverture,caracteristiques,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
0,NaN,NAN,9.782378e+12,"Shades - Tome 1, De sang et de chair- Grand F...",Angéline Richard,NaN,NaN,Alter Real éditions,NAN,46.314475,...,"5,99 €",https://products-images.di-static.com/image/ha...,Date de parution|30/08/2023|Editeur|Alter Real...,978-2-37812-870-8,Grand Format,Broché,368 pages,0.52 Kg,"14,8 cm × 21,0 cm × 0,0 cm",Imaginaire
1,NaN,NAN,9.782378e+12,Saraya Tome 5- Grand Format,Marylise,NaN,NaN,Alter Real éditions,NAN,46.314475,...,"5,99 €",https://products-images.di-static.com/image/ha...,Date de parution|30/08/2023|Editeur|Alter Real...,978-2-37812-908-8,Grand Format,Broché,188 pages,0.438 Kg,"14,8 cm × 21,0 cm × 0,0 cm",Imaginaire
2,RL,NON,9.782080e+12,Marchands de sable- Grand Format,Agnès Mathieu-Daudé,Romans français,F,Flammarion,FRANCE,46.603354,...,"14,99 €",https://products-images.di-static.com/image/ag...,Date de parution|30/08/2023|Editeur|Flammarion...,978-2-08-041169-3,Grand Format,Broché,336 pages,0.342 Kg,"15,0 cm × 22,0 cm × 2,1 cm",Littérature française
3,RL,PREMIER ROMAN,9.782383e+12,Au sol- Grand Format,Charlotte Milandri,Romans français,F,Equateurs,FRANCE,46.603354,...,"14,99 €",https://products-images.di-static.com/image/ch...,Date de parution|30/08/2023|Editeur|Equateurs|...,978-2-38284-605-6,Grand Format,Broché,248 pages,0.32 Kg,"14,0 cm × 20,3 cm × 2,0 cm",NaN
4,NaN,NAN,9.782226e+12,La saison des débutantes Tome 3- Grand Format,M-C Beaton,NaN,NaN,Albin Michel,NAN,46.314475,...,"9,99 €",https://products-images.di-static.com/image/m-...,Date de parution|30/08/2023|Editeur|Albin Mich...,978-2-226-46854-3,Grand Format,Broché,224 pages,0.21 Kg,"12,6 cm × 18,4 cm × 1,7 cm",NaN


RL                                                                  RL
PREMIER_ROMAN                                                      NON
EAN                                                    9782818058268.0
titre                                       Triste tigre- Grand Format
auteur                                                     Neige Sinno
TYPES                                                           Essais
GENRE                                                                F
Editeur                                                          P.O.L
PAYS                                                            FRANCE
latitude                                                     46.603354
longitude                                                     1.888334
CONTINENT                                                       EUROPE
Traduit de                                                          VF
Traduit par                                                          -
Date d

In [10]:
df_doublon.loc[df_doublon['EAN'] == 9782259316989, 'Dimensions'] = '22,2 x 14,4 cm'
#df_doublon.loc[df_doublon.RL == 'RL']['Nb. de pages'].value_counts()
display(df_doublon.loc[(df_doublon['EAN'] == 9782259316989) & (df_doublon['RL'] == 'RL')]['Dimensions'])
#df_doublon[['EAN','Dimensions']].to_csv('dimension_to_clean.csv', index = False)

1239    22,2 x 14,4 cm
Name: Dimensions, dtype: object

In [11]:
df_doublon.to_csv("liste_rl_total_sans_doublon.csv", sep=";", index=False)

In [114]:
df_doublon.to_pickle("liste_rl_total_sans_doublon.pkl")

In [9]:
test = pd.read_pickle("liste_rl_total_sans_doublon.pkl")
itest.loc[test['EAN'] == '3014,2 cm × 22,6 cm × 3,2 cm']['EAN'])

9782259316989

## Dico

dict_rl_23 = {"id" : "",
"nom": "",
"prenom" : "",
"nom_complet" : "",
"genre" : "",
"pays" : "",
"continent" : "",
"livre" : {"id": "",
	"titre": "",
	"ean" : "",
	"maison_edition" : "",
	"type" : "",
	"premier_roman" : "",
	"mois de parution" : "",
	"annee_parution" : "" ,
	"traduit_de" :"",
	"traduit_par" : "",
	"resume" : "test",
	"caracteristiques" : {
			"nombre_pages": "",
			"prix_indicatif" : "",
			"reliure" : "",
			"format" : ""},
	"autres_infos" : {
			"preface":"",
			"texte_autre":"",
			"illustration_photo_image":"",
			"oeuvre_original" : ""}},
"prix_litteraire" : 
	{"id_prix" : "",
	 "nom_prix" :"",
	 "annee" : "",
	 "premiere_selection" : "",
	 "deuxieme_selection" : "",
	 "troisieme_selection" : "",
	 "lauréat" : ""}}
dict_rl_23['livre']['resume']

In [3]:
df_doublon =  pd.read_pickle("liste_rl_total_test_pour_prix_litt.pkl")
df_doublon = df_doublon.drop(index = 0)#.reset_index(drop = True)
df_doublon.tail(60)

,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,Editeur,PAYS,latitude,...,prix_ebook,couverture,caracteristiques,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
1348,,NON,9782226458643,L'orageuse- Grand Format,Jessica-L Nelson,Romans français,F,Albin Michel,FRANCE,NaN,...,"Ebook14,99 €",https://products-images.di-static.com/image/je...,Date de parution|05/04/2023|Editeur|Albin Mich...,978-2-226-45864-3,Grand Format,Broché,409 pages,0.45 Kg,"14,0 cm × 20,5 cm × 3,0 cm",NaN
1349,RL,NON,9782226478566,Une philosophie du vin- Grand Format,Pierre-Yves Quiviger,Essais,M,Albin Michel,FRANCE,NaN,...,"Ebook12,99 €",https://products-images.di-static.com/image/pi...,Date de parution|30/08/2023|Editeur|Albin Mich...,978-2-226-47856-6,Grand Format,Broché,288 pages,0.288 Kg,"13,0 cm × 20,0 cm × 2,1 cm",NaN
1350,,NON,9782330160876,Le regard ramassé - Une anthologie de l'art m...,Paul Nizon,Essais,M,Actes Sud,FRANCE,NaN,...,"Ebook17,99 €",https://products-images.di-static.com/image/pa...,Date de parution|02/11/2022|Editeur|Actes Sud|...,978-2-330-16087-6,Grand Format,Broché,384 pages,0.368 Kg,"11,6 cm × 21,6 cm × 2,9 cm",NaN
1351,RL,NON,9791037509727,Le Piège Nord Stream- Grand Format,Marion Van Renterghem,Essais,F,Arènes (Editions Les),FRANCE,NaN,...,"Ebook17,99 €",https://products-images.di-static.com/image/ma...,Date de parution|14/09/2023|Editeur|Arènes (Ed...,979-10-375-0972-7,Grand Format,Broché,276 pages,0.342 Kg,"14,7 cm × 22,2 cm × 2,2 cm",NaN
1352,,NON,9782363083548,La vie sans histoire de James Castle- Poche,Luc Vezin,Romans français,M,Arléa,FRANCE,NaN,...,pas de prix ebook,https://products-images.di-static.com/image/lu...,Date de parution|05/10/2023|Editeur|Arléa|Coll...,978-2-36308-354-8,Poche,Broché,248 pages,0.22 Kg,"11,2 cm × 18,1 cm × 2,1 cm",Arléa-Poche
1353,,NON,9782363083234,"Mains, fils, ciseaux- Grand Format",Norbert Czarny,Romans français,M,Arléa,FRANCE,NaN,...,pas de prix ebook,https://products-images.di-static.com/image/no...,Date de parution|05/01/2023|Editeur|Arléa|Coll...,978-2-36308-323-4,Grand Format,Dos carré collé,172 pages,0.199 Kg,"12,6 cm × 20,8 cm × 1,5 cm",La rencontre
1354,,NON,9782251454177,Le génie de l'art indien- Grand Format,Vincent Lefèvre,Essais,M,Belles Lettres,FRANCE,NaN,...,"Ebook24,99 €",https://products-images.di-static.com/image/vi...,Date de parution|07/04/2023|Editeur|Belles Let...,978-2-251-45417-7,Grand Format,Broché,371 pages,0.51 Kg,"15,1 cm × 21,5 cm × 2,1 cm",NaN
1355,,NON,9782283036945,Tous immortels- Grand Format,Paul Pavlowitch,Essais,M,Buchet-Chastel,FRANCE,NaN,...,"Ebook14,99 €",https://products-images.di-static.com/image/pa...,Date de parution|02/02/2023|Editeur|Buchet-Cha...,978-2-283-03694-5,Grand Format,Broché,477 pages,0.465 Kg,"14,0 cm × 20,5 cm × 3,0 cm",Littérature française
1356,,NON,9782267051094,Les archives des sentiments- Grand Format,Peter Stamm,Romans étrangers,M,Christian Bourgois,SUISSE,NaN,...,"Ebook12,99 €",https://products-images.di-static.com/image/pe...,Date de parution|09/03/2023|Editeur|Christian ...,978-2-267-05109-4,Grand Format,Broché,192 pages,0.21 Kg,"12,0 cm × 20,0 cm × 1,7 cm",NaN
1357,,NON,9782080413642,La vitesse de l'ombre- Grand Format,Annie Le Brun,Essais,F,Flammarion,FRANCE,NaN,...,"Ebook15,99 €",https://products-images.di-static.com/image/an...,Date de parution|29/03/2023|Editeur|Flammarion...,978-2-08-041364-2,Grand Format,Broché,126 pages,0.288 Kg,"17,1 cm × 21,0 cm × 1,5 cm",NaN


In [5]:
dict_rl_23_bis = {}
#.reset_index(drop = True)
for i in df_doublon.index: 
    temp = {}
    temp["id"] = ""
    temp["nom"]= ""
    temp["prenom"] = ""
    temp["nom_complet"] = df_doublon['auteur'][i]
    temp["genre"] = df_doublon['GENRE'][i]
    temp["pays"] = df_doublon['PAYS'][i]
    temp["latitude"] = df_doublon['latitude'][i]
    temp["longitude"] = df_doublon['longitude'][i]
    temp["continent"] = df_doublon['CONTINENT'][i]
    temp["livre"] = {}
    temp["livre"]["id"] = "" 
    temp["livre"]["titre"]= df_doublon['titre'][i]
    temp["livre"]["ean"] = df_doublon['EAN'][i]
    temp["livre"]["maison_edition"] = df_doublon['Editeur'][i]
    temp["livre"]["type"] = df_doublon['TYPES'][i]
    temp["livre"]["premier_roman"] = df_doublon['PREMIER_ROMAN'][i]
    temp["livre"]["date de parution"] = df_doublon['Date de parution'][i]       
    temp["livre"]["mois de parution"] = df_doublon['MOIS'][i]
    temp["livre"]["annee_parution"] = df_doublon['ANNEE_PUBLICATION'][i]
    temp["livre"]["traduit_de"] = df_doublon['Traduit de'][i]
    temp["livre"]["traduit_par"] = df_doublon['Traduit par'][i]
    temp["livre"]["resume"] = ""
    temp["livre"]["caracteristiques"] = {}
    temp["livre"]["caracteristiques"]["nombre_pages"] = df_doublon['Nb. de pages'][i]
    temp["livre"]["caracteristiques"]["prix_indicatif"] = df_doublon['prix_gf'][i]
    temp["livre"]["caracteristiques"]["prix_ebook_indicatif"] = df_doublon['prix_ebook'][i]
    temp["livre"]["caracteristiques"]["reliure"] = df_doublon['Présentation'][i]
    temp["livre"]["caracteristiques"]["format"] = df_doublon['Format'][i]
    temp["livre"]["autres_infos"] = {}
    temp["livre"]["autres_infos"]["couverture"] = df_doublon['couverture'][i]
    temp["livre"]["autres_infos"]["preface"] = ""
    temp["livre"]["autres_infos"]["texte_autre"] = ""
    temp["livre"]["autres_infos"]["illustration_photo_image"] = ""
    temp["livre"]["autres_infos"]["oeuvre_original"] = ""
    temp["livre"]["prix_litteraire"] = {}
    #temp["livre"]["prix_litteraire"]["id_prix"] = ""
    #temp["livre"]["prix_litteraire"]["nom_prix"] = ""
    #temp["livre"]["prix_litteraire"]["annee"] = ""
    #temp["livre"]["prix_litteraire"]["premiere_selection"] = ""
    #temp["livre"]["prix_litteraire"]["deuxieme_selection"] = ""
    #temp["livre"]["prix_litteraire"]["troisieme_selection"] = ""
    #temp["livre"]["prix_litteraire"]["lauréat"] = ""
    dict_rl_23_bis[i] = temp
dict_rl_23_bis

{1348: {'id': '',
  'nom': '',
  'prenom': '',
  'nom_complet': 'Jessica-L Nelson',
  'genre': 'F',
  'pays': 'FRANCE',
  'latitude': nan,
  'longitude': nan,
  'continent': 'EUROPE',
  'livre': {'id': '',
   'titre': "L'orageuse- Grand Format",
   'ean': '9782226458643',
   'maison_edition': 'Albin Michel',
   'type': 'Romans français',
   'premier_roman': 'NON',
   'date de parution': '05/04/2023',
   'mois de parution': nan,
   'annee_parution': nan,
   'traduit_de': 'VF',
   'traduit_par': '-',
   'resume': '',
   'caracteristiques': {'nombre_pages': '409 pages',
    'prix_indicatif': '21,90 €',
    'prix_ebook_indicatif': 'Ebook14,99 €',
    'reliure': 'Broché',
    'format': 'Grand Format'},
   'autres_infos': {'couverture': 'https://products-images.di-static.com/image/jessica-l-nelson-l-orageuse/9782226458643-45x70-1.webp 1x,https://products-images.di-static.com/image/jessica-l-nelson-l-orageuse/9782226458643-45x70-2.webp 2x',
    'preface': '',
    'texte_autre': '',
    'illus

In [6]:
# Read dictionary pkl file
with open('dict_rl_23.pkl', 'rb') as fp:
    dict_rl_23 = pickle.load(fp)

def Merge(dict_1, dict_2):
	result = dict_1 | dict_2
	return result
	

dict_rl_final = Merge(dict_rl_23, dict_rl_23_bis)
print(dict_rl_final)

{0: {'id': '', 'nom': '', 'prenom': '', 'nom_complet': 'Angéline Richard', 'genre': nan, 'pays': 'NAN', 'latitude': 46.3144754, 'longitude': 11.0480288, 'continent': 'NAN', 'livre': {'id': '', 'titre': 'Shades  - Tome 1, De sang et de chair- Grand Format', 'ean': 9782378128708.0, 'maison_edition': 'Alter Real éditions', 'type': nan, 'premier_roman': 'NAN', 'date de parution': '2023-08-30', 'mois de parution': 'August', 'annee_parution': 2023.0, 'traduit_de': nan, 'traduit_par': nan, 'resume': '', 'caracteristiques': {'nombre_pages': '368 pages', 'prix_indicatif': '23,00 €', 'prix_ebook_indicatif': '5,99 €', 'reliure': 'Broché', 'format': 'Grand Format'}, 'autres_infos': {'couverture': 'https://products-images.di-static.com/image/hachette-shades/9782378128708-45x70-1.webp 1x,https://products-images.di-static.com/image/hachette-shades/9782378128708-45x70-2.webp 2x', 'preface': '', 'texte_autre': '', 'illustration_photo_image': '', 'oeuvre_original': ''}, 'prix_litteraire': {}}}, 1: {'id'

In [7]:
# Read dictionary pkl file
#with open('dict_rl_23.pkl', 'rb') as fp:
#    dict_rl_23 = pickle.load(fp)
#    #df_dict_rl_23 = pd.read_json(dict_rl_23, orient='index')   
#dict_rl_23#[379]["livre"]["prix_litteraire"]#['1']['nom_prix']= 'Prix Goncourt'
# save dictionary to person_data.pkl file
with open('dict_rl_final_23.pkl', 'wb') as fp:
    pickle.dump(dict_rl_final, fp)
    print('dictionary saved successfully to file')



dictionary saved successfully to file


In [9]:
df_dict_rl_23 = pd.DataFrame(dict_rl_23).T#,columns=["id","nom","prenom","nom_complet","genre","pays","latitude","longitude","continent","livre"])

df_dict_rl_23
df_dict_rl_23.loc[df_dict_rl_23['nom_complet'] == 'Neige Sinno')]['livre']

1289    {'id': '', 'titre': 'Triste tigre- Grand Forma...
Name: livre, dtype: object

In [46]:
dict_rl_23[165]["livre"]["prix_litteraire"].setdefault( '0',
{'id_prix': '2FE',
 'nom_prix': 'FEMINA',
 'id_prix_detail': '2FE1FR',
 'nom_prix_detail': 'FEMINA_FRANCAIS',
 'annee': '2023',
 'premiere_selection': 'OUI',
 'deuxieme_selection': 'NON',
 'troisieme_selection': 'NON',
 'lauréat': 'NON'}
)

#dict_rl_23[249]["livre"]["prix_litteraire"].setdefault( '1',
#{'id_prix': '2FE',
# 'nom_prix': 'FEMINA',
# 'id_prix_detail': '2FE2ET',
# 'nom_prix_detail': 'FEMINA_ETRANGER',
# 'annee': '2023',
# 'premiere_selection': 'OUI',
# 'deuxieme_selection': 'NON',
# 'troisieme_selection': 'NON',
# 'lauréat': 'NON'}
#)
#dict_rl_23[330]["livre"]["prix_litteraire"].update PAS DE SELECTION SUPP

{'id_prix': '2FE',
 'nom_prix': 'FEMINA',
 'id_prix_detail': '2FE1FR',
 'nom_prix_detail': 'FEMINA_FRANCAIS',
 'annee': '2023',
 'premiere_selection': 'OUI',
 'deuxieme_selection': 'NON',
 'troisieme_selection': 'NON',
 'lauréat': 'NON'}

|PRIX|PRIX_DETAIL|ID_PRIX|ID_PRIX_DETAIL|PREM_SELECTION|SEC_SELECTION|TROIS_SELECTION|RESULTAT|
|---|---|---|---|---|---|---|---|
FEMINA|FEMINA_FRANCAIS|2FE|2FE1FR|OUI|OUI|OUI|NON
FEMINA|FEMINA_LYCEENS|2FE|2FE4LY|OUI|NON|NON|NON


'0', {'id_prix': '2FE',
  'nom_prix': 'FEMINA',
  'id_prix_detail': '2FE4LY',
  'nom_prix_detail': 'FEMINA_LYCEENS',
  'annee': '2023',
  'premiere_selection': 'OUI',
  'deuxieme_selection': 'NON',
  'troisieme_selection': 'NON',
  'lauréat': 'NON'},
  
 '1', {'id_prix': '2FE',
  'nom_prix': 'FEMINA',
  'id_prix_detail': '2FE1FR',
  'nom_prix_detail': 'FEMINA_FRANCAIS',
  'annee': '2023',
  'premiere_selection': 'OUI',
  'deuxieme_selection': 'OUI',
  'troisieme_selection': 'NON',
  'lauréat': 'NON'}}



'2',
{'id_prix': '10RE',
 'nom_prix': 'RENAUDOT',
 'id_prix_detail': '10RE2ES',
 'nom_prix_detail': 'RENAUDOT_ESSAI',
 'annee': '2023',
 'premiere_selection': 'OUI',
 'deuxieme_selection': 'NON',
 'troisieme_selection': 'NON',
 'lauréat': 'NON'}


In [19]:
dict_rl_23[256]["livre"]["prix_litteraire"]#['1']['nom_prix']= 'Prix Goncourt'

{}

In [47]:
# save dictionary to person_data.pkl file
with open('dict_rl_23.pkl', 'wb') as fp:
    pickle.dump(dict_rl_23, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


## A paraître

In [ ]:
# url à scraper
url = 'https://www.decitre.fr/livres/litterature/romans/top-a-paraitre.html?dctr_format[1]=GF&order=dctr_publication_date&dir=asc&p='
#'https://www.decitre.fr/livres/litterature/romans/litterature-francaise/top-a-paraitre.html?dctr_format%5B0%5D=GF&dctr_publication_date%5B3%5D=%2B3months&dir=asc&order=dctr_publisher_sort&p='
#nb pages = 23
liste_rl = []

#your code
for nb in range(30):
    nb = nb+1
    trend = requests.get(url+str(nb)).content
    trend_soup = BeautifulSoup(trend,"html.parser")
    livre = trend_soup.findAll('li', attrs={"class" : "dct-product"})
    for x in livre :
        #for y in x.find('picture', attrs={"class" : "lozad"}):
        #    print (y['data-srcset'])
        url_data_livre = x.find('a')['href']
        #print("url : ", url_data_livre, "\n")

        data_livre = requests.get(url_data_livre).content
        data_livre_soup = BeautifulSoup(data_livre,"html.parser")

        #### infos du livre ###

        ## Catégories
        data_livre_cat = trend_soup.find('div', attrs={"id" : "main_breadcrumb",
                                      "class" : "breadcrumbs"}).findAll('span')
        liste_cat = [cat.get_text(strip=True) for cat in data_livre_cat]
        liste_cat_final = [c for c in liste_cat if c !='›']

        ## Titre
        data_livre_titre = data_livre_soup.find('h1', attrs={"class" : "product-title"}).get_text(strip=True)
        #print("titre : ", data_livre_titre, "\n")

        ### Auteur
        data_livre_auteur = data_livre_soup.find('a', attrs={"class" : "author author--main trackme"}).get_text(strip=True)
        #print("auteur : ", data_livre_auteur, "\n")


        ### prix grand format
        data_livre_prix_gf = data_livre_soup.find('div', attrs={"class" : "price fp-wide--margin-bottom"}).find('span').get_text(strip=True)#.findAll('span', attrs={"class" : "final-price"})
        #print("prix grand format : ", data_livre_prix_gf, "\n")

        ### prix ebook
        try :
            data_livre_prix_ebook = data_livre_soup.find('a', attrs={"class" : "ebook"}).get_text(strip=True)
            #print("prix ebook : ", data_livre_prix_ebook, "\n")
        except AttributeError:
            #print("pas de prix ebook", "\n")
            data_livre_prix_ebook = "pas de prix ebook"


        ## Couverture
        data_livre_couv = data_livre_soup.find('source', attrs={"class" : "lozad"})['data-srcset']
        #print("couv", data_livre_couv, "\n")


        ## Caractéristique
        data_livre_carac = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).get_text("|",strip=True)#.find_all('li',attrs={"class" :"information"}).get_text()
        #print("caracteristiques : ", data_livre_carac.get_text(),"\n")


        dic_rl = {'url' : url_data_livre,
           'categorie' : liste_cat_final,
           'titre':data_livre_titre,
           'auteur':data_livre_auteur,
           'prix_gf':data_livre_prix_gf,
           'prix_ebook':data_livre_prix_ebook,
            #'caracteristiques_bis' : temp_dict,
           'couverture':data_livre_couv,
           'caracteristiques':data_livre_carac}


        ## Caractéristique
        data_livre_carac_dict = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).findAll('li', attrs={"class" : "information"})
        for c in data_livre_carac_dict :
            try :
                k = c.find('span').get_text(strip=True)
                v = c.find('div', attrs={"class" :"value"}).get_text(strip=True)
            except AttributeError:
                print("NC")
            dic_rl[k]=v

        liste_rl.append(dic_rl)

In [ ]:
# création du dataFrame
df_rl_a_paraitre = pd.DataFrame(liste_rl)

# changement du format des dates de dd/mm/yyyy à YYYY-MM-DD
df_rl_a_paraitre["Date de parution"] = pd.to_datetime(df_rl_a_paraitre["Date de parution"], dayfirst=True)

# Ajout de colonnes
df_rl_a_paraitre = pd.concat((df_rl_a_paraitre,pd.DataFrame(columns=['TYPES','GENRE','PREMIER_ROMAN','MOIS','ANNEE_PUBLICATION','PAYS','CONTINENT','Traduit de','Traduit par'])), axis=1)

# export en CSV
df_rl_a_paraitre.to_csv('liste_rl.csv', sep=";", index = False)

# export au format pickle
df_rl_a_paraitre.to_pickle("liste_rl.pkl")
df_rl_a_paraitre = pd.read_pickle("liste_rl.pkl")

In [ ]:
#df_rl_tout_cat = pd.read_csv('liste_rl.csv', sep=";")#.drop(columns=['Unnamed: 0'])
df_rl_a_paraitre = pd.read_pickle("liste_rl.pkl")
df_rl_a_paraitre.head()

,url,categorie,titre,auteur,prix_gf,prix_ebook,couverture,caracteristiques,Date de parution,Editeur,...,Dimensions,TYPES,GENRE,PREMIER_ROMAN,MOIS,ANNEE_PUBLICATION,PAYS,CONTINENT,Traduit de,Traduit par
0,https://www.decitre.fr/livres/le-dernier-eunuq...,"[Livres, Littérature, Romans]",LE DERNIER EUNUQUE- Grand Format,Freidoune Sahebjam,"16,77 €",pas de prix ebook,https://products-images.di-static.com/image/fr...,Date de parution|01/12/1997|Editeur|Flammarion...,1997-12-01,Flammarion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.decitre.fr/livres/fourmis-mode-d-e...,"[Livres, Littérature, Romans]",Fourmis mode d'emploi- Grand Format,Bernard Werber,"5,95 €",pas de prix ebook,https://products-images.di-static.com/image/be...,Date de parution|01/02/1998|Editeur|Flammarion...,1998-02-01,Flammarion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.decitre.fr/livres/tapis-d-artistes...,"[Livres, Littérature, Romans]",Tapis d'artistes- Grand Format,Suzanne Day,"15,24 €",pas de prix ebook,https://products-images.di-static.com/image/ce...,Date de parution|01/01/1999|Editeur|Flammarion...,1999-01-01,Flammarion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.decitre.fr/livres/dominique-de-rou...,"[Livres, Littérature, Romans]",Dominique de Roux- Grand Format,Jean-Luc Barré,"17,00 €",pas de prix ebook,https://products-images.di-static.com/image/je...,Date de parution|01/01/1999|Editeur|Flammarion...,1999-01-01,Flammarion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.decitre.fr/livres/helena-rubinstei...,"[Livres, Littérature, Romans]",Héléna Rubinstein- Grand Format,Dumeige,"18,29 €",pas de prix ebook,https://products-images.di-static.com/image/ce...,Date de parution|01/01/1999|Editeur|Flammarion...,1999-01-01,Flammarion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Split string column into new columns
# Ajout de colonnes
df_rl_a_paraitre.columns

Index(['url', 'categorie', 'titre', 'auteur', 'prix_gf', 'prix_ebook',
       'couverture', 'caracteristiques', 'Date de parution', 'Editeur', 'ISBN',
       'EAN', 'Format', 'Présentation', 'Poids', 'Collection', 'Nb. de pages',
       'Dimensions', 'TYPES', 'GENRE', 'PREMIER_ROMAN', 'MOIS',
       'ANNEE_PUBLICATION', 'PAYS', 'CONTINENT', 'Traduit de', 'Traduit par'],
      dtype='object')

In [ ]:
df_rl_a_paraitre_2023 = df_rl_a_paraitre[df_rl_a_paraitre['Date de parution'] >= '2023-08-01']

df_rl_a_paraitre_2023.head()

,url,categorie,titre,auteur,prix_gf,prix_ebook,couverture,caracteristiques,Date de parution,Editeur,...,Dimensions,TYPES,GENRE,PREMIER_ROMAN,MOIS,ANNEE_PUBLICATION,PAYS,CONTINENT,Traduit de,Traduit par
60,https://www.decitre.fr/livres/batouala-9782810...,"[Livres, Littérature, Romans]",Batouala- Grand Format,René Maran,"12,90 €",pas de prix ebook,https://products-images.di-static.com/image/re...,Date de parution|31/08/2023|Editeur|Le Figaro ...,2023-08-31,Le Figaro Editions,...,"12,5 cm × 20,5 cm × 1,5 cm",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,https://www.decitre.fr/livres/juliette-9782221...,"[Livres, Littérature, Romans]",Juliette- Grand Format,Al malik Abd,"18,00 €","Ebook12,99 €",https://products-images.di-static.com/image/al...,Date de parution|31/08/2023|Editeur|Robert Laf...,2023-08-31,Robert Laffont,...,"13,4 cm × 20,2 cm × 1,4 cm",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,https://www.decitre.fr/livres/la-danse-des-dam...,"[Livres, Littérature, Romans]",La danse des damnées- Grand Format,Kiran Millwood Hargrave,"22,00 €","Ebook14,99 €",https://products-images.di-static.com/image/ki...,Date de parution|31/08/2023|Editeur|Robert Laf...,2023-08-31,Robert Laffont,...,"13,7 cm × 21,7 cm × 2,6 cm",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,https://www.decitre.fr/livres/une-journee-de-c...,"[Livres, Littérature, Romans]",Une journée de chien- Grand Format,Sander Kollaard,"19,00 €","Ebook12,99 €",https://products-images.di-static.com/image/sa...,Date de parution|31/08/2023|Editeur|Héloïse d'...,2023-08-31,Héloïse d'Ormesson,...,"14,2 cm × 20,8 cm × 1,8 cm",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,https://www.decitre.fr/livres/sous-les-strates...,"[Livres, Littérature, Romans]",Sous les strates- Grand Format,Lou Eve,"20,00 €","Ebook12,99 €",https://products-images.di-static.com/image/lo...,Date de parution|31/08/2023|Editeur|Les Escale...,2023-08-31,Les Escales (Editions),...,"14,4 cm × 22,9 cm × 2,2 cm",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_rl_a_paraitre.loc[df_rl_a_paraitre['titre'].str.contains('Jeanne')]

,url,categorie,titre,auteur,prix_gf,prix_ebook,couverture,caracteristiques,Date de parution,Editeur,...,Dimensions,TYPES,GENRE,PREMIER_ROMAN,MOIS,ANNEE_PUBLICATION,PAYS,CONTINENT,Traduit de,Traduit par


## Parus

In [ ]:
# url à scraper
url_parus = 'https://www.decitre.fr/livres/litterature/romans/nouveautes.html?dctr_format[0]=GF&dir=desc&order=dctr_publication_date&p='
#'https://www.decitre.fr/livres/litterature/romans/litterature-francaise/top-a-paraitre.html?dctr_format%5B0%5D=GF&dctr_publication_date%5B3%5D=%2B3months&dir=asc&order=dctr_publisher_sort&p='
#nb pages = 30
liste_rl_parus = []

#your code
for nb in range(30):
    nb = nb+1
    trend = requests.get(url_parus+str(nb)).content
    trend_soup = BeautifulSoup(trend,"html.parser")
    livre = trend_soup.findAll('li', attrs={"class" : "dct-product"})
    for x in livre :
        #for y in x.find('picture', attrs={"class" : "lozad"}):
        #    print (y['data-srcset'])
        url_data_livre = x.find('a')['href']
        #print("url : ", url_data_livre, "\n")

        data_livre = requests.get(url_data_livre).content
        data_livre_soup = BeautifulSoup(data_livre,"html.parser")

        #### infos du livre ###

        ## Catégories
        data_livre_cat = trend_soup.find('div', attrs={"id" : "main_breadcrumb",
                                      "class" : "breadcrumbs"}).findAll('span')
        liste_cat = [cat.get_text(strip=True) for cat in data_livre_cat]
        liste_cat_final = [c for c in liste_cat if c !='›']

        ## Titre
        data_livre_titre = data_livre_soup.find('h1', attrs={"class" : "product-title"}).get_text(strip=True)
        #print("titre : ", data_livre_titre, "\n")

        ### Auteur
        data_livre_auteur = data_livre_soup.find('a', attrs={"class" : "author author--main trackme"}).get_text(strip=True)
        #print("auteur : ", data_livre_auteur, "\n")


        ### prix grand format
        data_livre_prix_gf = data_livre_soup.find('div', attrs={"class" : "price fp-wide--margin-bottom"}).find('span').get_text(strip=True)#.findAll('span', attrs={"class" : "final-price"})
        #print("prix grand format : ", data_livre_prix_gf, "\n")

        ### prix ebook
        try :
            data_livre_prix_ebook = data_livre_soup.find('a', attrs={"class" : "ebook"}).get_text(strip=True)
            #print("prix ebook : ", data_livre_prix_ebook, "\n")
        except AttributeError:
            #print("pas de prix ebook", "\n")
            data_livre_prix_ebook = "pas de prix ebook"


        ## Couverture
        data_livre_couv = data_livre_soup.find('source', attrs={"class" : "lozad"})['data-srcset']
        #print("couv", data_livre_couv, "\n")


        ## Caractéristique
        data_livre_carac = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).get_text("|",strip=True)#.find_all('li',attrs={"class" :"information"}).get_text()
        #print("caracteristiques : ", data_livre_carac.get_text(),"\n")


        dic_rl_parus = {'url' : url_data_livre,
           'categorie' : liste_cat_final,
           'titre':data_livre_titre,
           'auteur':data_livre_auteur,
           'prix_gf':data_livre_prix_gf,
           'prix_ebook':data_livre_prix_ebook,
            #'caracteristiques_bis' : temp_dict,
           'couverture':data_livre_couv,
           'caracteristiques':data_livre_carac}


        ## Caractéristique
        data_livre_carac_dict = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).findAll('li', attrs={"class" : "information"})
        for c in data_livre_carac_dict :
            try :
                k = c.find('span').get_text(strip=True)
                v = c.find('div', attrs={"class" :"value"}).get_text(strip=True)
            except AttributeError:
                print("NC")
            dic_rl_parus[k]=v

        liste_rl_parus.append(dic_rl_parus)

In [ ]:
# création du dataFrame
df_rl_parus = pd.DataFrame(liste_rl_parus)

# changement du format des dates de dd/mm/yyyy à YYYY-MM-DD
df_rl_parus["Date de parution"] = pd.to_datetime(df_rl_parus["Date de parution"], dayfirst=True)

# Ajout de colonnes
df_rl_parus = pd.concat((df_rl_parus,pd.DataFrame(columns=['RL','TYPES','GENRE','PREMIER_ROMAN','MOIS','ANNEE_PUBLICATION','PAYS','CONTINENT','Traduit de','Traduit par'])), axis=1)

# Changement d'ordre des colonnes
df_rl_parus = df_rl_parus[['RL', 'PREMIER_ROMAN', 'EAN', 'titre', 'auteur', 'Date de parution', 'MOIS',
       'ANNEE_PUBLICATION', 'url', 'categorie','prix_gf', 'prix_ebook',
       'couverture', 'caracteristiques', 'Editeur', 'ISBN',
       'Format', 'Présentation', 'Nb. de pages', 'Poids', 'Dimensions',
       'Collection', 'TYPES', 'GENRE', 'PAYS', 'CONTINENT', 'Traduit de', 'Traduit par']]

df_rl_parus['MOIS'] = df_rl_parus['Date de parution'].apply(lambda x : x.month_name())
df_rl_parus['ANNEE_PUBLICATION'] = df_rl_parus['Date de parution'].apply(lambda x : x.year)

# export en CSV
df_rl_parus.to_csv('liste_rl_parus.csv', sep=";", index = False)

# export au format pickle
df_rl_parus.to_pickle("liste_rl_parus.pkl")
df_rl_parus = pd.read_pickle("liste_rl_parus.pkl")

In [ ]:
df_rl_parus.head()

,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,PAYS,CONTINENT,Traduit de,...,couverture,caracteristiques,Editeur,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
0,NaN,NaN,9782226484871,De l'inconvénient d'être Russe- Grand Format,Diana Filippova,NaN,NaN,NaN,NaN,NaN,...,https://products-images.di-static.com/image/di...,Date de parution|30/08/2023|Editeur|Albin Mich...,Albin Michel,978-2-226-48487-1,Grand Format,Broché,208 pages,0.296 Kg,"14,2 cm × 20,5 cm × 2,0 cm",NaN
1,NaN,NaN,9782246803829,"Comédie d'automne - (La vie poétique, 6 et fi...",Jean Rouaud,NaN,NaN,NaN,NaN,NaN,...,https://products-images.di-static.com/image/je...,Date de parution|30/08/2023|Editeur|Grasset|IS...,Grasset,978-2-246-80382-9,Grand Format,Broché,288 pages,0.308 Kg,"14,1 cm × 20,6 cm × 1,8 cm",NaN
2,NaN,NaN,9782378128708,"Shades - Tome 1, De sang et de chair- Grand F...",Angéline Richard,NaN,NaN,NaN,NaN,NaN,...,https://products-images.di-static.com/image/ha...,Date de parution|30/08/2023|Editeur|Alter Real...,Alter Real éditions,978-2-37812-870-8,Grand Format,Broché,368 pages,0.52 Kg,"14,8 cm × 21,0 cm × 0,0 cm",Imaginaire
3,NaN,NaN,9782378129088,Saraya Tome 5- Grand Format,Marylise,NaN,NaN,NaN,NaN,NaN,...,https://products-images.di-static.com/image/ha...,Date de parution|30/08/2023|Editeur|Alter Real...,Alter Real éditions,978-2-37812-908-8,Grand Format,Broché,188 pages,0.438 Kg,"14,8 cm × 21,0 cm × 0,0 cm",Imaginaire
4,NaN,NaN,9782080411693,Marchands de sable- Grand Format,Agnès Mathieu-Daudé,NaN,NaN,NaN,NaN,NaN,...,https://products-images.di-static.com/image/ag...,Date de parution|30/08/2023|Editeur|Flammarion...,Flammarion,978-2-08-041169-3,Grand Format,Broché,336 pages,0.342 Kg,"15,0 cm × 22,0 cm × 2,1 cm",Littérature française


In [ ]:
df_rl_parus.columns

Index(['RL', 'PREMIER_ROMAN', 'EAN', 'titre', 'auteur', 'TYPES', 'GENRE',
       'PAYS', 'CONTINENT', 'Traduit de', 'Traduit par', 'Date de parution',
       'MOIS', 'ANNEE_PUBLICATION', 'url', 'categorie', 'prix_gf',
       'prix_ebook', 'couverture', 'caracteristiques', 'Editeur', 'ISBN',
       'Format', 'Présentation', 'Nb. de pages', 'Poids', 'Dimensions',
       'Collection'],
      dtype='object')

## Fusion

### pour Google Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

Mounted at /content/drive


### pour Jupyter Notebook

### Pour Streamlit

In [62]:
%%writefile RL_2023.py

# 1.Importation des librairies nécessaires pour le script
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
#import scrapy
import re
import requests

import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np
import datetime as dt  #pour l'ajout de la date de l'extraction de données
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

import os

import time

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import streamlit as st
from streamlit_option_menu import option_menu


# 3.Setup de l'application Streamlit  - Streamlit webpage properties / set up the app with wide view preset and a title
st.set_page_config(page_title="Rentree Litteraire", page_icon="books", layout="wide")

st.write("RL_2023 GO")

#df_rl_total = pd.read_pickle("liste_rl_total.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/
#df_rl_total.tail(2)



#### Import pickel pour test

df_rl_total_test = pd.read_pickle("liste_rl_total_test.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

st.sidebar.success("Select an add method.")


row_nb = int(st.number_input('which row number ?'))

if st.button('drop index'):
    df_rl_total_test = df_rl_total_test.drop(index = row_nb)
    
if st.button("drop duplicate") :
    df_rl_total_test = df_rl_total_test[~df_rl_total_test.index.duplicated(keep='first')]

st.dataframe(df_rl_total_test.tail())


# 1. Find a book by the Editor name "Emmanuelle"
editeur = st.selectbox("Pick one", list(df_rl_total_test['Editeur'].unique()))

find_editeur = df_rl_total_test.loc[df_rl_total_test['Editeur'] == editeur] #, ['Genre']] = 'F'
st.dataframe(find_editeur)

# 2. choice of a method to add info for a book or add a book
#tab1, tab2, tab3 = st.tabs(["Ajout infos manquantes", "Ajout Automatique (url)", "Ajout Manuel"])
#tab1.write("this is tab 1")
#tab2.write("this is tab 2")


#with tab1:
#    with st.form("my_form"):
#        st.write("Inside the form")
#        nb_index = st.number_input('index')
#        prem_roman = st.selectbox('PREMIER ROMAN', ['OUI', 'NON'])
#        quel_genre = st.selectbox('genre', ['F', 'M'])
#        quel_pays = st.selectbox("Pays", list(df_rl_total_test['PAYS'].unique()), index = None)
#        quel_continent = st.selectbox("Continent", list(df_rl_total_test['CONTINENT'].unique()))
#        trad_de = st.selectbox("Traduit de", list(df_rl_total_test['Traduit de'].unique()), index = None)
#        trad_par = st.text_input('Traduit par')
#        st.form_submit_button('Submit my picks')
#
#    if quel_pays == None:
#        quel_pays = st.text_input("ajout pays")
#    
#    if trad_de == None :
#        trad_de = st.text_input("ajout langue")
#
#    #nb_index = float(nb_index)
#    df_rl_total_test.loc[int(nb_index),'RL'] = 'RL'
#    df_rl_total_test.loc[int(nb_index),'PREMIER_ROMAN'] = prem_roman #'PREMIER ROMAN'
#    df_rl_total_test.loc[int(nb_index),'TYPES'] = 'Romans étrangers'
#    df_rl_total_test.loc[int(nb_index),'GENRE'] = quel_genre
#    df_rl_total_test.loc[int(nb_index),'PAYS'] = quel_pays
#    df_rl_total_test.loc[int(nb_index),'CONTINENT'] = quel_continent #'EUROPE'
#    df_rl_total_test.loc[int(nb_index),'Traduit de'] = trad_de
#    df_rl_total_test.loc[int(nb_index),'Traduit par'] = trad_par


#with tab2 : 
#    #Pour ajout automatique
#    # url à scraper
#    with st.form("ajout auto"):
#        url_titre = st.text_input('url')
#        premier_roman = st.selectbox('PREMIER ROMAN', ['OUI', 'NON'])
#        types_livre = st.selectbox("type livre", ['Romans français' , 'Essais', 'Romans étrangers'])
#        genre = st.selectbox('genre', ['F', 'M'])
#        
#        pays = st.selectbox("Pays", list(df_rl_total_test['PAYS'].unique()))
#        continent = st.selectbox("Continent", list(df_rl_total_test['CONTINENT'].unique()))
#        langue_traduction = st.selectbox("Traduit de", list(df_rl_total_test['Traduit de'].unique()))
#        traducteurice = st.text_input('Traduit par')
#        
#        nb_index = st.number_input("index")
#        
#        submit_utl = st.form_submit_button('Submit my picks')
#    
#    liste_rl_titre = []
#
#    if pays == None :
#        pays = st.text_input("ajout pays")
#    
#    if traducteurice == None :
#        traducteurice = st.text_input("ajout langue")
#        
#    #your code
#    if submit_utl : 
#        data_livre = requests.get(url_titre).content
#        data_livre_soup = BeautifulSoup(data_livre,"html.parser")
#        
#        #### infos du livre ###
#        
#        ## Catégories
#        data_livre_cat = data_livre_soup.find('div', attrs={"id" : "main_breadcrumb",
#                                      "class" : "breadcrumbs"}).findAll('span')
#        liste_cat = [cat.get_text(strip=True) for cat in data_livre_cat]
#        liste_cat_final = [c for c in liste_cat if c !='›']
#        
#        ## Titre
#        data_livre_titre = data_livre_soup.find('h1', attrs={"class" : "product-title"}).get_text(strip=True)
#        #print("titre : ", data_livre_titre, "\n")
#        
#        ### Auteur
#        try :
#            data_livre_auteur = data_livre_soup.find('span', attrs={"class" : "author author--main"}).get_text(strip=True)
#        except :
#            data_livre_auteur = data_livre_soup.find('a', attrs={"class" : "author author--main trackme"}).get_text(strip=True)
#        #print("auteur : ", data_livre_auteur, "\n")
#        
#        
#        ### prix grand format
#        data_livre_prix_gf = data_livre_soup.find('div', attrs={"class" : "price fp-wide--margin-bottom"}).find('span').get_text(strip=True)#.findAll('span', attrs={"class" : "final-price"})
#        #print("prix grand format : ", data_livre_prix_gf, "\n")
#        
#        ### prix ebook
#        try :
#            data_livre_prix_ebook = data_livre_soup.find('a', attrs={"class" : "ebook"}).get_text(strip=True)
#            #print("prix ebook : ", data_livre_prix_ebook, "\n")
#        except AttributeError:
#            #print("pas de prix ebook", "\n")
#            data_livre_prix_ebook = "pas de prix ebook"
#        
#        
#        ## Couverture
#        data_livre_couv = data_livre_soup.find('source', attrs={"class" : "lozad"})['data-srcset']
#        #print("couv", data_livre_couv, "\n")
#        
#        
#        ## Caractéristique
#        data_livre_carac = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).get_text("|",strip=True)#.find_all('li',attrs={"class" :"information"}).get_text()
#        #print("caracteristiques : ", data_livre_carac.get_text(),"\n")
#        
#        
#        dic_rl_parus = {'url' : url_titre, #https://www.decitre.fr/livres/la-femme-a-la-valise-9782376650959.html
#            'categorie' : liste_cat_final,
#            'titre':data_livre_titre,
#            'auteur':data_livre_auteur,
#            'prix_gf':data_livre_prix_gf,
#            'prix_ebook':data_livre_prix_ebook,
#            #'caracteristiques_bis' : temp_dict,
#            'couverture':data_livre_couv,
#            'caracteristiques':data_livre_carac,
#            'RL' : 'RL',
#            'PREMIER_ROMAN' : premier_roman,
#            'TYPES' : types_livre,
#            'GENRE' : genre,
#            'PAYS' : pays,
#            'CONTINENT' : continent,
#            'Traduit de' : langue_traduction,
#            'Traduit par' : traducteurice,
#            }
#        
#        
#        ## Caractéristique
#        data_livre_carac_dict = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).findAll('li', attrs={"class" : "information"})
#        for c in data_livre_carac_dict :
#            try :
#                k = c.find('span').get_text(strip=True)
#                v = c.find('div', attrs={"class" :"value"}).get_text(strip=True)
#            except AttributeError:
#                print("NC")
#            dic_rl_parus[k]=v
#        
#        liste_rl_titre.append(dic_rl_parus)
#    
#        df_rl_total_test = pd.concat([df_rl_total_test,pd.DataFrame(liste_rl_titre, index=[int(nb_index)])])


#with tab3 :
#    with st.form("ajout_manuel"):
#        nb_index = st.number_input('index')
#        prem_roman = st.selectbox('PREMIER ROMAN', ['OUI', 'NON'])
#        quel_genre = st.selectbox('genre', ['F', 'M'])
#        quel_pays = st.selectbox("Pays", list(df_rl_total_test['PAYS'].unique()))
#        quel_continent = st.selectbox("Continent", list(df_rl_total_test['CONTINENT'].unique()))
#        trad_de = st.selectbox("Traduit de", list(df_rl_total_test['Traduit de'].unique()))
#        trad_par = st.text_input('Traduit par')
#        ean = st.text_input('EAN')
#        titre = st.text_input('titre')
#        auteur = st.text_input('auteur')
#        date_parution = st.text_input('Date de parution')
#        mois= st.selectbox('MOIS', ['August', 'September','October'])
#        annee_parution = st.number_input('ANNEE_PUBLICATION')
#        url = st.text_input('url')
#        categorie = st.text_input('categorie-liste')
#        prix_gf = st.text_input('prix_gf')
#        prix_ebook = st.text_input('prix_ebook ou pas de prix ebook')
#        couverture= st.text_input('url couv ou pas de visuel')
#        editeur = st.text_input('Editeur')
#        isbn = st.text_input('ISBN')
#        format = st.selectbox('genre', ['Grand Format', 'Poche'])
#        presentation = st.text_input('Présentation')
#        nb_pages = st.text_input('nb pages')
#        poids = st.text_input('Poids')
#        dimension= st.text_input('Dimensions')
#        collection = st.text_input('Collection') 
#        st.form_submit_button('Submit my picks')
#
#    if quel_pays == None :
#        quel_pays = st.text_input("ajout pays")
#    
#    if trad_de == None :
#        trad_de = st.text_input("ajout langue")
#    
#    dict_manuel = {'RL' : ['RL'],
#                   'PREMIER_ROMAN' : [prem_roman], #PREMIER ROMAN NON
#                    'TYPES' : ['Romans étrangers'], #Essais Romans étrangers Romans français
#                    'GENRE' : [quel_genre],
#                    'PAYS' : [quel_pays],
#                    'CONTINENT' : [quel_continent],
#                    'Traduit de' : [trad_de],
#                    'Traduit par' : [trad_par],
#                    'EAN' : [ean],
#                    'titre' : [titre],
#                    'auteur' : [auteur],
#                    'Date de parution' : [date_parution],
#                    'MOIS' : [mois], #August September October
#                    'ANNEE_PUBLICATION' : [annee_parution],
#                    'url' : [url],
#                    'categorie' : [[categorie]],
#                    'prix_gf' : [prix_gf + ' €'],
#                    'prix_ebook' : [prix_ebook], #pas de prix ebook 15,99 €
#                    'couverture' : [couverture], #pas de visuel
#                    'caracteristiques' : ['Date de parution|'+date_parution+'|editeur|'+editeur+'|ISBN|'+isbn+'|EAN|'+ean+'|Format|'+format+'|Présentation|'+presentation+'|Nb. de pages|'+nb_pages+' pages|Poids|'+poids+' kg|Dimensions|'+dimension+'|Collection|'+collection],
#                    'Editeur' : [editeur],
#                    'ISBN' : [isbn],
#                    'Format' : [format], #Grand Format Poche
#                    'Présentation' : [presentation],
#                    'Nb. de pages' : [nb_pages+'pages'],
#                    'Poids' : [poids+'kg'],
#                    'Dimensions' : [dimension], #14cm x 22cm
#                    'Collection' : [collection]}
#    
#    df1 = pd.DataFrame(dict_manuel, index=[int(nb_index)])
#    
#    df_rl_total_test = pd.concat([df_rl_total_test, df1])

#st.dataframe(df_rl_total_test(int(nb_index)))

# export au format pickle
df_rl_total_test.to_pickle("liste_rl_total_test.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

#df_rl_total_test = pd.read_pickle("liste_rl_total_test.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/
#st.dataframe(df_rl_total_test.tail(2))




Overwriting RL_2023.py


In [24]:
df_rl_total = pd.read_pickle("liste_rl_total_test.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/
df_rl_total.tail(2)



,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,Editeur,PAYS,CONTINENT,...,prix_ebook,couverture,caracteristiques,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
1346,RL,NON,9782378561802,Respire- Poche,Marielle Macé,Essais,F,Verdier Editions,FRANCE,EUROPE,...,"Ebook6,99 €",https://products-images.di-static.com/image/ma...,Date de parution|24/08/2023|Editeur|Verdier Ed...,978-2-37856-180-2,Poche,Broché,128 pages,0.09 Kg,"10,0 cm × 15,7 cm × 1,1 cm",La petite jaune
1347,RL,NON,9782356541222,Nouveau départ,May Ayim,Essais,F,YPSILON,ALLEMAGNE,EUROPE,...,pas de prix ebook,https://products-images.di-static.com/image/ma...,Date de parution|06/10/2023|Editeur|YPSILON|IS...,978-2-35654-122-2,NaN,Broché,168 pages,0.152 Kg,"11,5 cm × 17,0 cm × 1,0 cm",NaN


In [27]:


#df_rl_total.to_pickle("liste_rl_total_test.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/


ModuleNotFoundError: No module named 'geopy'

In [22]:
df_rl_total.loc[df_rl_total['Editeur'].str.contains("Emmanuelle") ==True] #, ['Genre']] = 'F'

,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,Editeur,PAYS,CONTINENT,...,prix_ebook,couverture,caracteristiques,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
53,RL,NON,9782490155804,Les femmes de Bidibidi- Grand Format,Charline Effah,Romans français,F,Collas Emmanuelle,FRANCE,EUROPE,...,"Ebook12,99 €",https://products-images.di-static.com/image/ch...,Date de parution|25/08/2023|Editeur|Collas Emm...,978-2-490155-80-4,Grand Format,Broché,228 pages,0.246 Kg,"12,6 cm × 19,2 cm × 2,1 cm",NaN
763,RL,NON,9782490155835,Nos destins sont liés- Grand Format,Walid Hajar Rachedi,Romans français,M,Collas Emmanuelle,FRANCE,EUROPE,...,"Ebook14,99 €",https://products-images.di-static.com/image/wa...,Date de parution|01/09/2023|Editeur|Collas Emm...,978-2-490155-83-5,Grand Format,Broché,NaN,0.36 Kg,"12,9 cm × 19,0 cm × 3,7 cm",NaN
770,NaN,NaN,9782490155729,Le Bunker de Tbilissi- Grand Format,Iva Pezuashvili,NaN,NaN,Collas Emmanuelle,NaN,NaN,...,"Ebook11,99 €",https://products-images.di-static.com/image/mi...,Date de parution|01/09/2023|Editeur|Collas Emm...,978-2-490155-72-9,Grand Format,Broché,156 pages,0.18 Kg,"12,8 cm × 19,1 cm × 2,1 cm",NaN


In [23]:
nb_index = int(input('index'))
prem_roman = input('PREMIER ROMAN')
quel_genre = input('genre')
quel_pays = input('Pays')
quel_continent = input('continent')
trad_de = input('langue traduite')
trad_par = input('traducteur')

df_rl_total.loc[nb_index,'RL'] = 'RL'
df_rl_total.loc[nb_index,'PREMIER_ROMAN'] = prem_roman #'PREMIER ROMAN'
df_rl_total.loc[nb_index,'TYPES'] = 'Romans étrangers'
df_rl_total.loc[nb_index,'GENRE'] = quel_genre
df_rl_total.loc[nb_index,'PAYS'] = quel_pays
df_rl_total.loc[nb_index,'CONTINENT'] = quel_continent #'EUROPE'
df_rl_total.loc[nb_index,'Traduit de'] = trad_de
df_rl_total.loc[nb_index,'Traduit par'] = trad_par

index 770
PREMIER ROMAN NON
genre M
Pays GEORGIE
continent EUROPE
langue traduite GEORGIEN
traducteur Marika Megrelishvili


#### Ajout Automatique

In [3]:
#Pour ajout automatique
# url à scraper
url_titre = input('url')
premier_roman = input("'PREMIER_ROMAN' ou 'NON'")
types_livre = input("'Romans français' ou 'Essais' 'Romans étrangers'")
genre = input("'M' ou 'F'")

pays = input("'pays ")
continent = input("continent ")
langue_traduction = input("Traduit de ")
traducteurice = input("Traduit par' ")

pour_index = input("index")

liste_rl_titre = []

#your code

data_livre = requests.get(url_titre).content
data_livre_soup = BeautifulSoup(data_livre,"html.parser")

#### infos du livre ###

## Catégories
data_livre_cat = data_livre_soup.find('div', attrs={"id" : "main_breadcrumb",
                              "class" : "breadcrumbs"}).findAll('span')
liste_cat = [cat.get_text(strip=True) for cat in data_livre_cat]
liste_cat_final = [c for c in liste_cat if c !='›']

## Titre
data_livre_titre = data_livre_soup.find('h1', attrs={"class" : "product-title"}).get_text(strip=True)
print("titre : ", data_livre_titre, "\n")

### Auteur
data_livre_auteur = data_livre_soup.find('a', attrs={"class" : "author author--main trackme"}).get_text(strip=True)
#print("auteur : ", data_livre_auteur, "\n")


### prix grand format
data_livre_prix_gf = data_livre_soup.find('div', attrs={"class" : "price fp-wide--margin-bottom"}).find('span').get_text(strip=True)#.findAll('span', attrs={"class" : "final-price"})
#print("prix grand format : ", data_livre_prix_gf, "\n")

### prix ebook
try :
    data_livre_prix_ebook = data_livre_soup.find('a', attrs={"class" : "ebook"}).get_text(strip=True)
    #print("prix ebook : ", data_livre_prix_ebook, "\n")
except AttributeError:
    #print("pas de prix ebook", "\n")
    data_livre_prix_ebook = "pas de prix ebook"


## Couverture
data_livre_couv = data_livre_soup.find('source', attrs={"class" : "lozad"})['data-srcset']
#print("couv", data_livre_couv, "\n")


## Caractéristique
data_livre_carac = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).get_text("|",strip=True)#.find_all('li',attrs={"class" :"information"}).get_text()
#print("caracteristiques : ", data_livre_carac.get_text(),"\n")


dic_rl_parus = {'url' : url_titre,
    'categorie' : liste_cat_final,
    'titre':data_livre_titre,
    'auteur':data_livre_auteur,
    'prix_gf':data_livre_prix_gf,
    'prix_ebook':data_livre_prix_ebook,
    #'caracteristiques_bis' : temp_dict,
    'couverture':data_livre_couv,
    'caracteristiques':data_livre_carac,
    'RL' : 'RL',
    'PREMIER_ROMAN' : premier_roman,
    'TYPES' : types_livre,
    'GENRE' : genre,
    'PAYS' : pays,
    'CONTINENT' : continent,
    'Traduit de' : langue_traduction,
    'Traduit par' : traducteurice,
    }


## Caractéristique
data_livre_carac_dict = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).findAll('li', attrs={"class" : "information"})
for c in data_livre_carac_dict :
    try :
        k = c.find('span').get_text(strip=True)
        v = c.find('div', attrs={"class" :"value"}).get_text(strip=True)
    except AttributeError:
        print("NC")
    dic_rl_parus[k]=v

liste_rl_titre.append(dic_rl_parus)


df_rl_total = pd.concat([df_rl_total,pd.DataFrame(liste_rl_titre, index=[pour_index])])
df_rl_total.tail(2)

url https://www.decitre.fr/livres/l-odeur-des-clementines-grillees-9782367271163.html
'PREMIER_ROMAN' ou 'NON' N
'Romans français' ou 'Essais' 'Romans étrangers' N
'M' ou 'F' N
'pays  N
continent  N
Traduit de  N
Traduit par'  N
index 9999


titre :  L'Odeur des clémentines grillées- Grand Format 



AttributeError: 'NoneType' object has no attribute 'get_text'

In [9]:
data_livre = requests.get("https://www.decitre.fr/livres/l-odeur-des-clementines-grillees-9782367271163.html").content
data_livre_soup = BeautifulSoup(data_livre,"html.parser")
#print(data_livre_soup)

#### infos du livre ###

## Catégories
data_livre_cat = data_livre_soup.find('div', attrs={"id" : "main_breadcrumb",
                              "class" : "breadcrumbs"}).findAll('span')
liste_cat = [cat.get_text(strip=True) for cat in data_livre_cat]
liste_cat_final = [c for c in liste_cat if c !='›']

## Titre
data_livre_titre = data_livre_soup.find('h1', attrs={"class" : "product-title"}).get_text(strip=True)
print("titre : ", data_livre_titre, "\n")

### Auteur
data_livre_auteur = data_livre_soup.find('span', attrs={"class" : "author author--main"}).get_text(strip=True)
print("auteur : ", data_livre_auteur, "\n")


### prix grand format
data_livre_prix_gf = data_livre_soup.find('div', attrs={"class" : "price fp-wide--margin-bottom"}).find('span').get_text(strip=True)#.findAll('span', attrs={"class" : "final-price"})
#print("prix grand format : ", data_livre_prix_gf, "\n")

### prix ebook
try :
    data_livre_prix_ebook = data_livre_soup.find('a', attrs={"class" : "ebook"}).get_text(strip=True)
    #print("prix ebook : ", data_livre_prix_ebook, "\n")
except AttributeError:
    #print("pas de prix ebook", "\n")
    data_livre_prix_ebook = "pas de prix ebook"


## Couverture
data_livre_couv = data_livre_soup.find('source', attrs={"class" : "lozad"})['data-srcset']
#print("couv", data_livre_couv, "\n")


## Caractéristique
data_livre_carac = data_livre_soup.find('ul', attrs={"class" :"informations-container"}).get_text("|",strip=True)#.find_all('li',attrs={"class" :"information"}).get_text()
#print("caracteristiques : ", data_livre_carac.get_text(),"\n")


titre :  L'Odeur des clémentines grillées- Grand Format 



AttributeError: 'NoneType' object has no attribute 'get_text'

#### Export pickel et CSV

In [24]:
# export en CSV
df_rl_total.to_csv('liste_rl_total.csv', sep=";", index = False) #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

# export au format pickle
df_rl_total.to_pickle("liste_rl_total.pkl") #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

pd.read_pickle("liste_rl_total.pkl").tail(2) #/content/drive/MyDrive/Data4Good/rentree_litteraire/2023/

,RL,PREMIER_ROMAN,EAN,titre,auteur,TYPES,GENRE,Editeur,PAYS,CONTINENT,...,prix_ebook,couverture,caracteristiques,ISBN,Format,Présentation,Nb. de pages,Poids,Dimensions,Collection
1263,RL,OUI,9782385290313,Le rêve de la couturière- Grand Format,Bianca Pitzorno,Romans étrangers,F,Charleston éditions,ITALIE,EUROPE,...,"Ebook16,99 €",https://products-images.di-static.com/image/bi...,Date de parution|19/09/2023|Editeur|Charleston...,978-2-38529-031-3,Grand Format,Dos carré collé,265 pages,0.336 Kg,"14,7 cm × 23,0 cm × 2,2 cm",NaN
1264,RL,NON,9782824622606,Un dernier été avant la guerre,Merryn Allingham,Romans étrangers,F,City Editions,ANGLETERRE,EUROPE,...,"12,99 €",https://products-images.di-static.com/image/me...,Date de parution|(2023-09-06|Editeur|City Edit...,978-2-8246-2260-6,Grand Format,Dos carré collé,382 pages,0.516 kg,"14,4 cm × 22,6 cm × 2,9 cm",Romans


In [ ]:
df_rl_total.columns

Index(['RL', 'PREMIER_ROMAN', 'EAN', 'titre', 'auteur', 'TYPES', 'GENRE',
       'PAYS', 'CONTINENT', 'Traduit de', 'Traduit par', 'Date de parution',
       'MOIS', 'ANNEE_PUBLICATION', 'url', 'categorie', 'prix_gf',
       'prix_ebook', 'couverture', 'caracteristiques', 'Editeur', 'ISBN',
       'Format', 'Présentation', 'Nb. de pages', 'Poids', 'Dimensions',
       'Collection'],
      dtype='object')

#### Ajout Manuel

In [20]:
# pour ajout titre manquant manuel

# Append one or more rows of another dataframe
#
df1 = pd.DataFrame({'RL' : ['RL'],
'PREMIER_ROMAN' : ['NON'], #PREMIER ROMAN NON
'TYPES' : ['Romans étrangers'], #Essais Romans étrangers Romans français
'GENRE' : ['F'],
'PAYS' : ['ANGLETERRE'],
'CONTINENT' : ['EUROPE'],
'Traduit de' : ['ANGLAIS'],
'Traduit par' : ['Marion Boclet'],
'EAN' : ['9782824622606'],
'titre' : ["Un dernier été avant la guerre"],
'auteur' : ['Merryn Allingham'],
'Date de parution' : ['2023-09-06'],
'MOIS' : ['September'], #August September October
'ANNEE_PUBLICATION' : [2023],
'url' : ['https://www.decitre.fr/livres/l-heritiere-de-summerhayes-9782824622606.html'],
'categorie' : [['Livres', 'Romance', 'Romance historique']],
'prix_gf' : [' 20,00 €'],
'prix_ebook' : ['12,99 €'], #pas de prix ebook 15,99 €
'couverture' : ['https://products-images.di-static.com/image/merryn-allingham-l-heritiere-de-summerhayes/9782824622606-475x500-2.webp'], #pas de visuel
'caracteristiques' : ['Date de parution|(2023-09-06|Editeur|City Editions|ISBN|978-2-8246-2260-6 |EAN|9782824622606|Format|Grand Format|Présentation|Dos carré collé |Nb. de pages|382 pages|Poids|0.516 kg|Dimensions|14,4 cm × 22,6 cm × 2,9 cm|Collection|Romans'],
'Editeur' : ['City Editions'],
'ISBN' : ['978-2-8246-2260-6 '],
'Format' : ['Grand Format'], #Grand Format Poche
'Présentation' : ['Dos carré collé'],
'Nb. de pages' : ['382 pages'],
'Poids' : ['0.516 kg '],
'Dimensions' : ['14,4 cm × 22,6 cm × 2,9 cm'], #14cm x 22cm
'Collection' : ['Romans']
}, index=[1264])
#
# Append a dataframe
#

df_rl_total = pd.concat([df_rl_total, df1])




***

## Prix littéraire 2023
|PRIX|PRIX_DETAIL|
|---|---|
|Prix Wepler|Prix Wepler-Fondation La Poste|
|RENAUDOT|RENAUDOT_ROMAN|
|RENAUDOT|RENAUDOT_ESSAI|
|RENAUDOT|Prix Renaudot des lycéens|
|Prix Goncourt|Prix Goncourt des Lycéens|
|Prix Goncourt|Prix Goncourt|
|Prix Medicis|Prix Medicis_FR|
|Prix Medicis|Prix Medicis_ET|
|Prix André Malraux|Prix André Malraux_ESSAI_ART|
|Prix André Malraux|Prix André Malraux_LITT_ENGAGEE|
|Prix Interallié|Prix Interallié|
|Grand Prix de l'Académie Française|Grand Prix de l'Académie Française|
|Prix Décembre|Prix Décembre|
| Prix Alain Spiess|Prix Alain Spiess|
|FEMINA|FEMINA_FRANCAIS|
|FEMINA|FEMINA_ETRANGER|
|FEMINA|FEMINA_LYCEENS|
|FEMINA|FEMINA_ESSAI|
|Prix Medicis|Prix Medicis_ESSAIS|
|Prix de Flore|Prix de Flore|


### Prix Décembre

Le Canadien Kevin Lambert a remporté mardi 31 octobre le prix Décembre pour Que notre joie demeure (éditions du Nouvel Attila), une fiction sur la chute d’une architecte accusée de chasser les pauvres de Montréal. Voici les auteurs qui étaient en lice :

- Kevin Lambert, Que notre joie demeure (Nouvel Attila)
- Geneviève Damas, Strange (Grasset)
- Laure Murat, Proust, roman familial (Robert Laffont)
- Neige Sinno, Triste Tigre (P.O.L)

### Prix Femina

Neige Sinno a remporté lundi 6 novembre le prix Femina du roman français pour Triste tigre (éditions POL), récit de l’inceste dont elle a été victime enfant, et essai sur les violences sexuelles. Le prix Femina du roman étranger a été décerné à l’Américaine Louise Erdrich, pour La Sentence (Albin Michel), histoire d’une libraire amérindienne confrontée aux fantômes du passé et au racisme du présent. Enfin, le jury a attribué son prix de l’essai au chercheur français Hugo Micheron, spécialiste du jihadisme, pour La Colère et l’Oubli (Gallimard). L’essai se penche sur les mouvements islamistes radicaux et violents en Europe après l’effondrement de l’organisation Etat islamique en Syrie. Voici la liste des derniers auteurs qui étaient en lice :

Romans français
- Jean-Baptiste Andrea, Veiller sur elle (L’Iconoclaste)
- Pierric Bailly, La Foudre (P.O.L)
- Guy Boley, À ma sœur et unique (Grasset)
- Agnès Mathieu-Daudé, Marchands de sable (Flammarion)
- Neige Sinno, Triste Tigre (P.O.L)

Romans étrangers 
- Louise Erdrich, La Sentence (Albin Michel)
- Lidia Jorge, Misericordia (Métailié)
- Han Kang, Impossibles adieux (Grasset)
- Maggie O’Farrell, Le Portrait de mariage (Belfond)
- Robert Seethaler, Le Café sans nom (Sabine Wespieser)

Essais
- Négar Djavadi, La Dernière Place (éd. Stock)
- Hugo Micheron, La Colère et l’Oubli (éd. Gallimard)
- Nathalie Piegay, 3 Nanas. Saint Phalle, Bourgeois, Messager (éd. Seuil)
- Perrine Simon-Nahum, Sagesse du politique (éd. L’Observatoire)
- Marion van Renterghem, Le Piège Nord Stream (éd. Les Arènes)
- Joëlle Zask, Se tenir quelque part sur la Terre (éd. Premier parallèle)
Cet article est régulièrement mis à jour.

### Prix Goncourt

Le prix Goncourt a été décerné mardi 7 novembre à Jean-Baptiste Andrea pour Veiller sur elle (éditions L’Iconoclaste), une histoire d’amour au temps du fascisme en Italie.

Voici les auteurs qui étaient en lice sur la troisième liste :
- Jean-Baptiste Andrea, Veiller sur elle (L’Iconoclaste)
- Gaspard Kœnig, Humus (L’Observatoire)
- Éric Reinhardt, Sarah, Susanne et l’écrivain (Gallimard)
- Neige Sinno, Triste Tigre (P.O.L)

### Prix Renaudot

Le prix Renaudot a été attribué mardi 7 novembre à Ann Scott pour son roman Les Insolents (éditions Calman-Lévy), sur une quadragénaire quittant Paris. Les jurés du Renaudot ont attribué leur prix de l’essai à De Gaulle, une vie, de Jean-Luc Barré. Voici la liste des derniers auteurs qui étaient en lice :



#### Sélection de romans 
- François Bégaudeau, L’Amour (Verticales)
- Éric Chacour, Ce que je sais de toi (Philippe Rey)
- Sorj Chalandon, L’Enragé (Grasset)
- Lilia Hassaine, Panorama (Gallimard)
- Gaspard Koenig, Humus (L’Observatoire)
- Maria Pourchet, Western (Stock)
- Alexis Salatko, Jules et Joe (Denoël)
- Ann Scott, Les Insolents (Calmann-Lévy)
- Fred Vargas, Sur la dalle (Flammarion)

#### Sélection d’essais 
- Nathacha Appanah, La Mémoire délavée (Mercure de France)
- Jean-Luc Barré, De Gaulle, une vie : l’homme de personne (Grasset)
- Hubert Haddad, L’art et son miroir (Zulma)
- Bruno de Cessole, Le Sceptre et la Plume (Perrin)
- Paul Pavlowitch, Tous immortels (Buchet-Chastel)
- Cédric Sapin-Dufour, Son odeur après la pluie (Stock)

### Prix de Flore

Maria Pourchet a remporté mercredi 8 novembre le prix de Flore pour Western (éditions Stock), dans lequel elle observe la fuite d’un acteur accusé de violences. Voici les auteurs qui étaient en lice :

- François Bégaudeau, L’amour (Verticales)
- Clément Camar-Mercier, Le roman de Jeanne et Nathan (Actes-Sud)
- Jean Carrère, Perdre (Allia)
- Eva Ionesco, La bague au doigt (Robert Laffont)
- Maria Pourchet, Western (Stock)

### Prix Médicis

Le prix Médicis Étranger 2023 a été décerné jeudi 9 novembre à Han Kang pour Impossibles Adieux, traduit du coréen, ex-aequo avec Lidia Jorge, autrice de Misericordia. Le prix de l’essai a été remis à Laure Murat pour Proust, roman familial. Enfin Kevin Lambert double la mise puisqu’il avait déjà reçu le prix Décembre et a obtenu le prix du roman francophone pour Que notre joie demeure. Voici les auteurs qui étaient en lice :

#### Romans francophones

- Dominique Barbéris, Une façon d’aimer (éd. Gallimard)
- Salma El Moumni, Adieu Tanger (éd. Grasset)
- Kevin Lambert, Que notre joie demeure (éd. Le Nouvel Attila)
- Lisette Lombé, Eunice (éd. du Seuil)
- Richard Morgiève, La fête des mères (éd. Joëlle Losfeld)
- Eric Reinhardt, Sarah, Suzanne et l’écrivain (éd. Gallimard)
- Neige Sinno, Triste tigre (éd. P.O.L.)
- Elisa Shua Dusapin, Le vieil incendie (éd. Zoé)

#### Romans étrangers 

- Nina Allan, Conquest (éd. Tristram)
- Hila Blum, Comment aimer sa fille (éd. Robert Laffont)
- David Grann, Les Naufragés du Wager (éd. du Sous-sol)
- Lidia Jorge, Misericordia (éd. Métailié)
- Han Kang, Impossibles adieux (éd. Grasset)
- László Krasznahorkai, Le Baron Weinckein est de retour (éd. Cambourakis)
- Kim de L’Horizon, Hêtre pourpre (éd. Julliard)
- Robert Seethaler, Le Café sans nom (éd. Sabine Wespieser)
- Chris de Stoop, Le Livre de Daniel (éd. Globe)

#### Essais 

- Daniel Andler, Intelligence artificielle, intelligence humaine : La double énigme (éd. Gallimard)
- Nathacha Appanah, La Mémoire délavée (éd. Mercure de France)
- Robert Bober, Il y a quand même dans la rue des gens qui passent (éd. P.O.L.)
- Benoît Chantre, René Girard. Biographie (éd. Grasset)
- Ariane Chemin, Ne réveille pas les enfants (éd. du sous-sol)
- Marie Favereau, La Horde. Comment les mongols ont changé le monde (éd. Perrin)
- Hélène Frappat, Le Gaslighting ou l’art de faire taire les femmes (éd. de L’Observatoire)
- Anouche Kunth, Au bord de l’effacement. Sur les pas d’exilés arméniens dans l’entre-deux-guerres (éd. La Découverte)
- Valérie Mréjen, La Jeune artiste (éd. P.O.L.)
- Laure Murat, Proust, roman familial (éd. Robert Laffont)
- Nathalie Piégay, 3 Nanas : Saint-Phalle, Bourgeois, Messager (éd. du Seuil)
- Judith Schalansky, Inventaire de choses perdues (éd. Ypsilon)
- Camille de Toledo, Une histoire du vertige (éd. Verdier)

### Prix Interallié

Le prix sera décerné le 22 novembre.

- Jean-Baptiste Andrea, Veiller sur elle, éd. L’Iconoclaste
- Claire Conruyt, Pour qui s’avance dans la nuit, éd. L’Observatoire
- Régis Franc, Je vais bien, éd. Les Presses de la Cité
- Gaspard Koenig, Humus, éd. L’Observatoire
- David Le Bailly, Hôtel de la Folie, éd. du Seuil
- Jessica Nelson, L’orageuse, éd. Albin Michel
- Louis-Henri de la Rochefoucauld, Les petits farceurs, éd. Robert Laffont
- Charlie Roquin, Les Maîtres de Bayreuth, éd. Le Cherche Midi
- Bertrand de Saint Vincent, Une certaine désinvolture, Ed. du Rocher

### Prix Goncourt des lycéens

Le prix sera décerné le 23 novembre.

- Mokhtar Amoudi, Les conditions idéales (Gallimard)
- Jean-Baptiste Andrea, Veiller sur elle (L’Iconoclaste)
- Dominique Barbéris, Une façon d’aimer (Gallimard)
- Vincent Delecroix, Naufrage, (Gallimard)
- Cécile Desprairies, La propagandiste (Seuil)
- Émilie Frèche, Les amants du Lutetia (Albin Michel)
- Dorothée Janin, La révolte des filles perdues (Stock)
- Gaspard Koening, Humus (L’Observatoire)
- Kevin Lambert, Que notre joie demeure (Le Nouvel Attila)
- Akira Mizubayashi, Suite inoubliable (Gallimard)
- Laure Murat, Proust, roman familial (Robert Laffont)
- Léonor de Récondo, Le grand feu (Grasset)
- Éric Reinardt, Sarah, Susanne et l’écrivain (Gallimard)
- Antoine Sénanque, Croix de cendre (Grasset)
- Neige Sinno, Triste tigre (P.O.L)
- Jean-Philippe Toussaint, L’échiquier (Éditions de Minuit)

In [13]:
rentree_litteraire_df = pd.DataFrame(columns = ["PRIX","PRIX_DETAIL","SCORE","PREM_SELECTION","SEC_SELECTION",
 "TROIS_SELECTION","RESULTAT","AUTEUR","TITRE","EDITEUR","IMAGE",
 "IMAGE_URL","ISBN","RL_2022","GENRE"])

{"AUTEUR": Mokhtar Amoudi, Les Conditions idéales, Gallimard},
{"AUTEUR": Gaëlle Bélem, Le Fruit le plus rare, Gallimard},
{"AUTEUR": Clément Camar-Mercier, Le Roman de Jeanne et Nathan, Actes sud},
{"AUTEUR": Eric Chacour, Ce que je sais de toi, Philippe Rey},
{"AUTEUR": Sorj Chalandon, L’Enragé, Grasset},
{"AUTEUR": Nicolas Chemla, L’Abîme, Le Cherche-Midi},
{"AUTEUR": Lionel Duroy, Mes pas dans leurs ombres, Mialet/Barrault},
{"AUTEUR": Lilia Hassaine, Panorama, Gallimard},
{"AUTEUR": Gaspard Koenig, Humus, L’Observatoire},
{"AUTEUR": Jessica L. Nelson, L’Ombrageuse, Albin Michel}
{"AUTEUR": Maria Pourchet, Western, Stock
{"AUTEUR": Thomas B. Reverdy, Le Grand Secours, Flammarion}
{"AUTEUR": Alexis Salatko, Jules et Jo, Denoël}
{"AUTEUR": Ann Scott, Les Insolents, Calmann-Lévy}
{"AUTEUR": Antoine Sénanque, Croix de cendre, Grasset}
{"AUTEUR": Fred Vargas, Sur la dalle, Flammarion}

rentree_litteraire_df

,PRIX,PRIX_DETAIL,SCORE,PREM_SELECTION,SEC_SELECTION,TROIS_SELECTION,RESULTAT,AUTEUR,TITRE,EDITEUR,IMAGE,IMAGE_URL,ISBN,RL_2022,GENRE


## Autres

In [ ]:
import PyPDF2

import pypdf

import pdfplumber

In [ ]:
from pdf2image import convert_from_path

for img in convert_from_path("LH_RL_2023.pdf", 300):
    print(img)
    txt = tool.image_to_string(img,
                               lang=lang,
                               builder=pyocr.builders.TextBuilder())




In [ ]:
brew install poppler

In [ ]:
pdf_file = open('LH_RL_2023.pdf', 'rb')
read_pdf = PyPDF2.PdfReader(pdf_file)

In [ ]:
page = reader.pages[0]
page

In [ ]:
page = read_pdf.getPage(0)
page_content = page.extractText()
print(page_content.encode('utf-8'))

---

In [ ]:
import spacy#.cli
#spacy.cli.download("fr_core_news_md")
nlp = spacy.load("fr_core_news_md")

## * ** ***Debut Tuto SpaCy*** ** *
Cet objet nous informe de l’utilisation/qualification de chaque mot (pour plus de détails allez dans l’aide de SpaCy):

    text: Le texte/mot original
    lemma_: la forme de base du mot (pour un verbe conjugué par exemple on aura son infinitif)
    pos_: Le tag part-of-speech (détail ici)
    tag_: Les informations détaillées part-of-speech (détail ici)
    dep_: Dépendance syntaxique (inter-token)
    shape: format/pattern
    is_alpha: Alphanumérique ?
    is_stop: Le mot fait-il partie d’une Stop-List ?
    etc.

In [ ]:
doc = nlp('Demain je travaille à la maison')

for token in doc:
    print("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\t{8}".format(
        token.text,
        token.idx,
        token.lemma_,
        token.is_punct,
        token.is_space,
        token.shape_,
        token.pos_,
        token.tag_,
        token.ent_type_
    ))

In [ ]:
doc_p = nlp("Demain je travaille à la maison. Je vais pouvoir faire du NLP")
for sent in doc_p.sents:
    print(sent)

In [ ]:
doc_ner = nlp("Demain je travaille en France chez Tableau")
for ent in doc_ner.ents:
    print(ent.text, ent.label_)

In [ ]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True, options={'distance': 130})

## Fin Tuto

## 1) Extraction texte du fichier PDF

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# pip install requests - > request for get the pdf
# pip install BeautifulSoup - > for parse the html and find all url hrf with ".pdf" final
import PyPDF2

import io
import pdfreader
from pdfreader import PDFDocument, SimplePDFViewer

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        print(num_pages)

        for page_number in [38,40,42,44, 46,48,50,52,53,54,56,57,58,60,61,
                            62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,
                           81,82,83,84,85]:
            page = reader.pages[page_number]
            #text += page.extract_text()
            pageObj = pdfReader.getPage(page)
            print(pageObj.extractText().split('\n'))

    return text

if __name__ == "__main__":
    pdf_file_path = "LH_RL_2023.pdf"
    extracted_text = extract_text_from_pdf(pdf_file_path)
    print(extracted_text)

In [ ]:
with open("LH_RL_2023.pdf") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

soup = BeautifulSoup("<html>a web page</html>", 'html.parser')

#soup.find("div", {"class": "percentage good"}).get_text(strip=True)

In [ ]:
lste_lh=[]
for page in [0,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55]:
  # creating a page object
    print(page)
    pageObj = pdfReader.getPage(page)
    lste_lh.append(pageObj.extractText().split('\n'))



In [ ]:
print(pageObj.extractText().split('\n'))

In [ ]:
#df_lh = pd.DataFrame(pageObj.extractText().split('\n'))
#df_lh[0].str.split(',', expand=True)

In [ ]:
df_lh = pd.DataFrame(lste_lh)

In [ ]:
df_lh

## 2) Enrichissement fichier
Au final, il m'a fallu retraité l'extraction car le scrapping n'a pas permis un rendu propre et ordonné.
Je poursuis le nettoyage de données à l'aide de python pour ensuite passer à l'enrichissement des données.

### Etape initiale où je nettoie les données

In [ ]:
rl22=pd.read_csv("recap_page_20220801.csv")
rl22=rl22[['ISBN','TYPES', 'TITRE', 'AUTEUR*E','1ER ROMAN', 'EDITEUR', 'MOIS','ANNEE_PUBLICATION', 'PREFACE', 'TEXTE_AUTRES',
       'ILLUSTRATIONS_PHOTO_IMAGES', 'OEUVRE ORIGINALE', 'Traduit de',
       'Traduit par', 'RESUME','NB PAGES', 'FORMAT', 'RELIURE', 'PRIX INDICATIF', ]]
## passer les prix au format float
prix_to_replace=[" €","€"]
for tr in prix_to_replace:
    rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].apply(lambda x : x.replace(tr,""))
    rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].apply(lambda x : x.replace(",","."))
rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].astype("float")

## passer les nb de pages au format int
nc_to_replace=["NC","nc"]
for nc in nc_to_replace:
    rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace(nc,""))
rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace('2 vol. (380)',"380"))
rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace('144,0 €',"144"))
rl22["NB PAGES"] = [int(p) if len(p)>0 else 0 for p in rl22["NB PAGES"]]

rl22.head(3)

### Etape où les données sont nettoyées.
Ajout de la colonne GENRE fait sur Excel

In [ ]:
rl22=pd.read_csv("recap_page_20220825.csv")
#rl22=rl22[['ISBN','TYPES', 'TITRE', 'AUTEUR*E','1ER ROMAN', 'EDITEUR', 'MOIS','ANNEE_PUBLICATION', 'PREFACE', 'TEXTE_AUTRES',
#       'ILLUSTRATIONS_PHOTO_IMAGES', 'OEUVRE ORIGINALE', 'Traduit de',
#       'Traduit par', 'RESUME','NB PAGES', 'FORMAT', 'RELIURE', 'PRIX INDICATIF', ]]
### passer les prix au format float
prix_to_replace=[" €","€"]
for tr in prix_to_replace:
#    rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].apply(lambda x : x.replace(tr,""))
    rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].apply(lambda x : x.replace(",","."))
rl22["PRIX INDICATIF"] = rl22["PRIX INDICATIF"].astype("float")


#colonne 1er roman : remplacer les NaN par des vides
rl22["1ER ROMAN"] = ["NVX ROMAN" if (pr!="PREMIER ROMAN" and t!="Essais") else pr for pr,t in zip(rl22["1ER ROMAN"],rl22["TYPES"])]
#
### passer les nb de pages au format int
#nc_to_replace=["NC","nc"]
#for nc in nc_to_replace:
#    rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace(nc,""))
#rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace('2 vol. (380)',"380"))
#rl22["NB PAGES"]  = rl22["NB PAGES"].apply(lambda x : x.replace('144,0 €',"144"))
#rl22["NB PAGES"] = [int(p) if len(p)>0 else 0 for p in rl22["NB PAGES"]]
#
rl22.tail(3)

In [ ]:
rl22.info()

J'ai maintenant un fichier avec les colonnes suivantes :

| Nom colonne | type données | Description |
|:--|:--|:--|
|ISBN|INTEGER|ISBN de l'ouvrage <i>(peut servir d'id)</i>||
|TYPES|SRTING|type de texte (essai, littérature française, littérature étrangère)|
|complet|SRTING|description complète du titre <i>(non affiché dans le dataframe)</i>|
|TITRE|SRTING|titre de l'ouvrage|
|AUTEUR*E|SRTING|prénom, nom de l'auteur*e|
|GENRE|SRTING|F femme, M pour homme, Mixte si collectif ou plusieurs noms facilement identifiable en termes de genre|
|1ER ROMAN|SRTING|*PREMIER ROMAN* si c'est un premier roman, *vide* si un essai, sinon *NVX ROMAN*|
|EDITEUR|SRTING|nom de la maison d'édition|
|MOIS|SRTING|mois de parution|
|ANNEE_PUBLICATION|INTEGER|année de publication <i>(ici 2022)</i>|
|PREFACE|SRTING|rempli (nom du préfaceur) s'il y en a une, sinon vide|
|TEXTE_AUTRES|SRTING|autres indications relatives à l'ouvrage (ex : textes présentés/sous la directino de/...)|
|ILLUSTRATIONS_PHOTO_IMAGES|SRTING|rempli (nom du contributeur) si indiqué, sinon vide|
|OEUVRE ORIGINALE|SRTING|rempli si indiqué, sinon vide|
|Traduit de|SRTING|rempli (nom du contributeur) si indiqué, sinon VF|
|Traduit par|SRTING|rempli (nom du contributeur) si indiqué, sinon "-"|
|RESUME|SRTING|résumé de l'ouvrage|
|NB PAGES|INTEGER|nb de pages indicatif de l'ouvrage|
|FORMAT|SRTING|format indicatif de l'ouvrage (largeur x hauteur) en cm|
|RELIURE|SRTING|type de reliure de l'ouvrage (broché, relié, coffret)|
|PRIX INDICATIF|FLOAT|prix indicatif de l'ouvrage|

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



#### Nombre d'ouvrages total

In [ ]:
#plt.bar(reliure_df.RELIURE,reliure_df.ISBN)
#fig = px.bar(reliure_df, x="ISBN",  orientation='h', color="RELIURE")

fig_nb_livre = go.Figure(go.Indicator(
    mode = "number", #mode = "number+delta",
    value = rl22.ISBN.count(),
    number = {'font_color':'white','suffix': ""},
    title = {'font_color':'white',"text": "Rentrée littéraire 2022<br><span style='font-size:0.8em;color:white'>Nombre d'ouvrages total</span><br><span style='font-size:0.8em;color:gray'></span>"},
    #delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig_nb_livre.update_layout(paper_bgcolor = "white",
                 autosize=True, #width=250, height=250,
                 images= [dict(
                    source="https://images.unsplash.com/photo-1594312915251-48db9280c8f1?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2070&q=80",
                    xref="paper", yref="paper",
                    x=1, y=0.3,
                    sizex=1.5, sizey=1.5,
                    xanchor="right", #['right', 'center', 'left']
                    yanchor="bottom", #['top', 'middle', 'bottom']
                    sizing="contain", #['fill', 'contain', 'stretch']
                    layer="below")]) #['below', 'above'])

fig_nb_livre.show()

#### Analyse par types d'ouvrages

In [ ]:
type_df = rl22[['ISBN',"TYPES"]].groupby("TYPES").count().sort_values('ISBN', ascending=False).reset_index()
type_df

In [ ]:
fig = go.Figure()

fig.add_trace(
go.Indicator(
    #mode = "number", #mode = "number+delta",
    value = int(type_df.ISBN.loc[type_df.TYPES=="Romans français"]),
    #number = {'suffix': " ouvrages"},
    title = {"text": "Romans français"},#<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>
    #delta = {'position': "top", 'reference': 320},
    domain = {'row': 0, 'column': 0}))


fig.add_trace(
go.Indicator(
    #mode = "number", #mode = "number+delta",
    value = int(type_df.ISBN.loc[type_df.TYPES=="Romans étrangers"]),
    #number = {'suffix': " ouvrages"},
    title = {"text": "Romans étrangers"},#<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>
    #delta = {'position': "top", 'reference': 320},
    domain = {'row': 0, 'column': 1}))


fig.add_trace(
go.Indicator(
    #mode = "number", #mode = "number+delta",
    value = int(type_df.ISBN.loc[type_df.TYPES=="Essais"]),
    #number = {'suffix': " ouvrages"},
    title = {"text": "Essais"},#<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>
    #delta = {'position': "top", 'reference': 320},
    domain = {'row': 0, 'column': 2}))

fig.update_layout(
    grid = {'rows': 1, 'columns': 3, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
                'mode' : "number",
                #'number': {'suffix': " ouvrages"}
    }]
                         }},
                 autosize=True)



#fig = go.Figure(go.Indicator(
#    mode = "number", #mode = "number+delta",
#    value = reliure_df.ISBN.sum(),
#    number = {'suffix': " ouvrages"},
#    title = {"text": "Relié<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>"},
#    #delta = {'position': "top", 'reference': 320},
#    domain = {'x': [0, 1], 'y': [0, 1]}))
#
#fig.update_layout(paper_bgcolor = "pink")

fig.show()

In [ ]:
# create data - Nb de dossier par nombre de retard
size_of_groups=type_df['ISBN']
names=[t for t in type_df.TYPES]

# Create a pieplot
plt.pie(size_of_groups, colors=['darkcyan','palevioletred','darkorange'],autopct='%1.1f%%',pctdistance=0.85) #labels=names,

# add a circle at the center to transform it in a donut chart
my_circle=plt.Circle( (0,0), 0.5, color='white')

# Adding Title of chart
plt.title('Nb de livres par type d\'ouvrages \n (565 livres)')

# Add Legends
plt.legend(names, loc="lower right", title="Nb de causes de retard \n pour un dossier", bbox_to_anchor=(1.7, 0.4))

p=plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

#### Nombre de pages & Prix indicatif

Je regarde les statistiques classiques pour les variables numériques que sont le nb de pages et le prix indicatif.

In [ ]:
rl22["NB PAGES"].value_counts().sort_index(ascending=True)

In [ ]:
rl22[['NB PAGES','PRIX INDICATIF']].describe()

#### nb pages :
<pre><code>rl22["NB PAGES"].value_counts().sort_index(ascending=True)</code></pre>

|nb de pages|nb d'ouvrages|
|:--|:-:|
|0|165|
|48|3|
|52|1|
|65|1|
|80|1|
|...|...|
|650|1|
|700|1|
|736|1|
|750|2|
|876|1|

- le nb de pages moyen est de 192. 13 livres ont exactement ce nb de pages.
<pre><code>rl22.loc[rl22["NB PAGES"]==192]</code></pre>

|ISBN|TYPES|TITRE|AUTEUR*E|EDITEUR|
|:--|:--|:--|:--|:--|
|9782380822496|Essais|Tout garder|Carole Allamand|ANNE CARRIÈRE|
|9782413042938|Romans étrangers|Au-delà|Julia Alvarez|LA CROISÉE|
|9782383611257|Romans étrangers|Pourquoi tu revenais tous les étés ?|Belén Lopez Peiro|GLOBE|
|9782493213143|Romans étrangers|Ca n’arrive qu’aux autres|Bettina Wilpert|NOUVEL ATTILA|
|9782330169237|Romans français|Moi en plus beau|Guillaume Le Touze|ACTES SUD|
|9782365331159|Romans français|La fugue Thérémine|Emmanuel Villin|ASPHALTE|
|9782714497918|Romans français|Les chairs impatientes|Marion Roucheux|BELFOND|
|9782490251650|Romans français|Une saison avec Luce|Henri Raczymow|CANOË|
|9782490251667|Romans français|Je suis né comme un mourant|Didier Dumont|CANOË|
|9782359841572|Romans français|Le netsuke|Thomas Lavachery|ESPERLUÈTE|
|9782080239204|Romans français|L’homme qui danse|Victor Jestin|FLAMMARION|
|9782882507488|Romans français|La nuit des pères|Gaëlle Josse|NOIR SUR BLANC|
|9782268106724|Romans français|À l’ombre de la cité Rimbaud|Halimata Fofana|ÉDITIONS DU ROCHER|


- J'ai 165 titres sans nb de pages indiqués. Pour l'instant, ils sont mis à 0. Je complèterais éventuellement par la suite.
- les titres les plus court font 48 pages. Il s'agit de 3 essais :
<pre><code>rl22.loc[rl22["NB PAGES"]==48]</code></pre>

|ISBN|TYPES|TITRE|AUTEUR*E|EDITEUR|
|:--|:--|:--|:--|:--|
|9782917504611|Essais|C’est la faute à Démosthène|Dominique Grandmont|BARQUE|
|9782371583054|Essais|Journal|Michel Magnol|ONDE|
|9782492352089|Essais|Grammaire pour cesser d’exister|Amélie Durand|LE SABOT|

- ***Le cœur ne cède pas*** est l'ouvrage le plus long de cette rentrée littéraire, il contient 876 pages :

|ISBN|TYPES|TITRE|AUTEUR*E|EDITEUR|
|:--|:--|:--|:--|:--|
|9782080247377 |Romans français|Le cœur ne cède pas|Grégoire Bouillier|FLAMMARION|

#### prix :
- Le prix moyen d'un livre de la rentrée est de 19,2€
- J'ai repéré un titre dont le prix minimal est de 2€. En fait il s'agit d'un essai qui paraïtra dans la collection Folio, dont le prix minimal peut être de 2€ donc ce n'est pas une erreur.
<pre><code>rl22.loc[rl22["PRIX INDICATIF"]==rl22["PRIX INDICATIF"].min()]</code></pre>

|ISBN|TYPES|TITRE|AUTEUR*E|EDITEUR|
|:--|:--|:--|:--|:--|
|9782072885990|Essais|Petit éloge de la Belgique|Grégoire Polet|GALLIMARD|

![](https://www.gallimard.fr/var/storage/images/product/fd6/product_9782072885990_195x320.jpg)<br>

- Il vous faudra débourser 55€ pour obtenir le livre le plus cher.
<pre><code>rl22.loc[rl22["PRIX INDICATIF"]==rl22["PRIX INDICATIF"].min()]</code></pre>

|ISBN|TYPES|TITRE|AUTEUR*E|EDITEUR|
|:--|:--|:--|:--|:--|
|9782252046869|Essais|Œuvres et inédits : édition critique intégrale. 19, Sur le concept d’histoire|Walter Benjamin|KLINCKSIECK|

- Fun fact:
Il vous faudrait 10855.6€ pour acquérir la totalité des livres de la Rentrée Littéraire.
<pre><code>rl22["PRIX INDICATIF"].sum()</code></pre>

In [ ]:
fig_pages = go.Figure(go.Indicator(
    mode = "number", #mode = "number+delta",
    value = rl22["NB PAGES"].sum(),
    number = {'suffix': "Pages"},
    title = {"text": "Fun fact :<br><span style='font-size:0.8em;color:gray;'>pour acquérir la totalité des livres de la Rentrée Littéraire</span><br><span style='font-size:0.8em;color:gray'> il vous faudra débourser</span>"},
    #delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig_pages.update_layout(paper_bgcolor = "white",
                 autosize=False, width=550, height=250)
fig_pages.show()

In [ ]:
fig_pages = go.Figure(go.Indicator(
    mode = "number", #mode = "number+delta",
    value = rl22["NB PAGES"].mean(),
    number = {'suffix': "Pages"},
    title = {"text": "Fun fact :<br><span style='font-size:0.8em;color:gray'>pour acquérir la totalité des livres de la Rentrée Littéraire</span><br><span style='font-size:0.8em;color:gray'> il vous faudra débourser</span>"},
    #delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig_pages.update_layout(paper_bgcolor = "white",
                 autosize=False, width=550, height=250)
fig_pages.show()

In [ ]:
fig_prix = go.Figure(go.Indicator(
    mode = "number", #mode = "number+delta",
    value = rl22["PRIX INDICATIF"].sum(),
    number = {'suffix': "€"},
    title = {"text": "Fun fact :<br><span style='font-size:0.8em;color:gray'>pour acquérir la totalité des livres de la Rentrée Littéraire</span><br><span style='font-size:0.8em;color:gray'> il vous faudra débourser</span>"},
    #delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig_prix.update_layout(paper_bgcolor = "white",
                 autosize=False, width=550, height=250)
fig_prix.show()

**Graph : Répartition des prix**

In [ ]:
import plotly.express as px

fig_hist_prix = px.histogram(rl22, x="PRIX INDICATIF",
                   title='Répartition par prix',
                   labels={'ISBN':'NB OUVRAGES'},
                   histnorm='')


fig_hist_prix.show()

In [ ]:
import plotly.express as px

fig_prix_type = px.histogram(rl22, x="PRIX INDICATIF",
                   title="Répartition par prix par Type",
                   labels={'ISBN':'NB OUVRAGES'},
                   histnorm='',
                  color='TYPES',
                  marginal="violin") # can be `box`, `violin`,'rug', hover_data=rl22.columns)


fig_prix_type.show()

In [ ]:
import plotly.express as px

fig_prix_prem_rom = px.histogram(rl22, x="PRIX INDICATIF",
                   title='Répartition par prix',
                   labels={'ISBN':'NB OUVRAGES'},
                   histnorm='percent',
                  color='TYPES',
                  pattern_shape="1ER ROMAN")


fig_prix_prem_rom.show()

In [ ]:
import seaborn as sns
sns.distplot(rl22["PRIX INDICATIF"], kde=False, color='red', bins=30)
plt.title('Répartition prix indicatif', fontsize=18)
plt.xlabel('prix indicatif', fontsize=14)
plt.ylabel('nb de livres', fontsize=14)

In [ ]:
prix_max = rl22.loc[rl22["PRIX INDICATIF"]==rl22["PRIX INDICATIF"].max()]
prix_max

In [ ]:
nb_pages = rl22[['NB PAGES',"PRIX INDICATIF","TYPES","ISBN"]].groupby(['NB PAGES',"PRIX INDICATIF","TYPES"]).count().sort_values('NB PAGES', ascending=False).reset_index()
#pd.DataFrame(rl22["NB PAGES"].value_counts().sort_index(ascending=True))
nb_pages

In [ ]:
fig_prix_page = px.scatter(nb_pages, x='PRIX INDICATIF', y='NB PAGES',size='ISBN',
                 color='TYPES', log_x=True, size_max=60,color_discrete_sequence=px.colors.qualitative.Alphabet)
fig_prix_page.show()

In [ ]:
#import requests
#from ipywidgets import Image
#
#Image(value=requests.get('https://octodex.github.com/images/yaktocat.png').content)
#

#### Les premiers romans en question

In [ ]:
prem_rom = rl22[['ISBN',"1ER ROMAN"]].groupby(["1ER ROMAN"]).count()
prem_rom

In [ ]:
prem_rom_graph = rl22[['ISBN',"1ER ROMAN"]].groupby(["1ER ROMAN"]).count().reset_index()
fig_prem_rom = px.pie(values=prem_rom_graph.ISBN, names=prem_rom_graph['1ER ROMAN'],
                      hole=0.3,
                     color_discrete_sequence=px.colors.sequential.Rainbow)

fig_prem_rom.update_layout(
                paper_bgcolor = "white",
                autosize=False, width=550, height=250,
                title = {"text": "FUN FACT : <br><span style='font-size:0.8em;color:gray'>21% des ouvrages sont des 1er Romans</span><br><span style='font-size:0.8em;color:gray'></span>"},
         )
fig_prem_rom.show()

In [ ]:
prem_rom_type = rl22[['ISBN',"TYPES","1ER ROMAN"]].groupby(["TYPES","1ER ROMAN"]).count().sort_values('ISBN', ascending=False).reset_index().set_index('TYPES')
prem_rom_type

In [ ]:
fig_prem_rom_type = px.bar(prem_rom_type, x="ISBN", y="1ER ROMAN", orientation='h', color=prem_rom_type.index,hover_data=['ISBN'],
                          title='1er roman vs Nouveau roman')
fig_prem_rom_type.show()

#### Analyse par types de reliure

In [ ]:
reliure_df = rl22[['ISBN',"RELIURE"]].groupby("RELIURE").count().sort_values('ISBN', ascending=False).reset_index()
reliure_df

In [ ]:
# create data - Nb de dossier par nombre de retard
size_of_groups=reliure_df['ISBN']
names=[t for t in reliure_df.RELIURE]

# Create a pieplot
plt.pie(size_of_groups, colors=['darkcyan','palevioletred','darkorange'],autopct='%1.1f%%',pctdistance=0.85) #labels=names,

# add a circle at the center to transform it in a donut chart
my_circle=plt.Circle( (0,0), 0.0, color='white')

# Adding Title of chart
plt.title('Nb de livres par type d\'ouvrages \n (565 livres)')

# Add Legends
plt.legend(names, loc="lower right", title="Nb de causes de retard \n pour un dossier", bbox_to_anchor=(1.7, 0.4))

p=plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

Overview

In this tutorial we introduce a new trace named "Indicator". The purpose of "indicator" is to visualize a single value specified by the "value" attribute. Three distinct visual elements are available to represent that value: number, delta and gauge. Any combination of them can be specified via the "mode" attribute. Top-level attributes are:

    value: the value to visualize
    mode: which visual elements to draw
    align: how to align number and delta (left, center, right)
    domain: the extent of the figure

Then we can configure the 3 different visual elements via their respective container:

    number is simply a representation of the number in text. It has attributes:
    valueformat: to format the number
    prefix: a string before the number
    suffix: a string after the number
    font.(family|size): to control the font

"delta" simply displays the difference between the value with respect to a reference. It has attributes:

    reference: the number to compare the value with
    relative: whether that difference is absolute or relative
    valueformat: to format the delta
    (increasing|decreasing).color: color to be used for positive or decreasing delta
    (increasing|decreasing).symbol: symbol displayed on the left of the delta
    font.(family|size): to control the font
    position: position relative to νmber

(either top, left, bottom, right)
prefix: a string to appear before the delta
suffix: a string to appear after the delta

In [ ]:
reliure_df

#### Analyse par éditeurs

In [ ]:
editeur = rl22[['ISBN',"EDITEUR"]].groupby(["EDITEUR"]).count().sort_values('ISBN',ascending=False)
editeur

In [ ]:
editeur_type = rl22[['ISBN',"TYPES","EDITEUR"]].groupby(["TYPES","EDITEUR"]).count().pivot_table(values='ISBN', index=['EDITEUR'],columns=['TYPES'], aggfunc=np.sum).sort_values('Romans français',ascending=False).reset_index()
editeur_type

#### Analyse par genre

In [ ]:
genre = rl22[['ISBN',"TYPES","GENRE"]].groupby(["GENRE"]).count().sort_values('ISBN',ascending=False).reset_index()
genre

In [ ]:
genre_type = rl22[['ISBN',"TYPES","GENRE"]].groupby(["GENRE","TYPES"]).count().sort_values('ISBN',ascending=False).reset_index().pivot_table(values='ISBN', index=['GENRE'],columns=['TYPES'], aggfunc=np.sum).reset_index()
genre_type

**Traduction** :
On dénombre 154 traduction dont 145 en littérature étrangère.<br>
78 sont en anglais (américain, anglais - uk/irlande/Ecosse/Ile Maurice)
<pre><code> traduction[['ISBN',"split"]].loc[(traduction["split"]!='nc')&(traduction["Traduit de"]!='VF')&(traduction["TYPES"]=='Romans étrangers')].groupby("split").count().sort_values('ISBN',ascending=False)</pre></code>

|split|ISBN|
|:--|:--|
|Américain|49|
|Anglais|29|
|Espagnol|12|
|Italien|10|
|Allemand|9|
|Suédois|5|
|Portugais|4|
|Japonais|3|
|Russe|3|
|Norvégien|2|
|Persan|2|
|Chinois|2|
|Arabe|1|
|Italien|1|
|Croate|1|
|Macédonien|1|
|Hébreu|1|
|Néerlandais|1|
|Hongrois|1|
|Polonais|1|
|Hollandais|1|
|Grec|1|
|Catalan|1|
|Turc|1|

traduction.ISBN.sum()

In [ ]:
traduction=rl22[['ISBN',"TYPES","Traduit de"]]
traduction['split']=[x.split(' (')[0] for x in traduction["Traduit de"]]
traduction_gp = traduction[['ISBN',"split"]].loc[(traduction["split"]!='nc')&(traduction["Traduit de"]!='VF')&(traduction["TYPES"]=='Romans étrangers')].groupby("split").count().sort_values('ISBN',ascending=False)
traduction_gp


In [ ]:
# Import the required library
from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

##location = geolocator.geocode("england")
#
#print("The latitude of the location is: ", location.latitude)
#print("The longitude of the location is: ", location.longitude)
#
#traduction['lat']=[geolocator.geocode(countries_dict[t]).latitude for t in traduction['Traduit de']]
#traduction['lon']=[geolocator.geocode(countries_dict[t]).longitude for t in traduction['Traduit de']]
#

In [ ]:
countries_dict = {"Américain":"usa",
"Anglais":"england",
"Italien":"italia",
"Italien ":"italia",
"Espagnol":"spain",
"Allemand":"germany",
"Suédois":"sweden",
"Anglais (Irlande)":"ireland",
"Portugais":"portugal",
"Russe":"russia",
"Japonais":"japan",
"Chinois":"china",
"Norvégien":"norway",
"Persan":"iran",
"Polonais ":"Poland",
"Polonais":"Poland",
"Hébreu":"israel",
"Néerlandais (Pays-Bas) ":"nederlands",
"Macédonien":"macedonia",
"Grec":"greece",
"Hongrois":"hungary",
"Hollandais":"nederlands",
"Allemand (Autriche)":"austria",
"Espagnol (Cuba)":"cuba",
"Espagnol (Colombie)":"colombia",
"Espagnol (Argentine)":"argentina",
"Croate":"croatia",
"Catalan":"spain",
"Arabe (Soudan)":"sudan",
"Anglais (pays de Galles)":"wales",
"Anglais (Ile Maurice)":"mauritius",
"Créole mauricien":"mauritius",
"Anglais (Ecosse)":"Scotland",
"Turc":"turkey",
"VF":"france" }

countries_df = pd.DataFrame.from_dict(countries_dict,orient='index',columns=['pays']).reset_index()
countries_df = countries_df.rename(columns={'index':'langue'})
countries_df['lat']=[geolocator.geocode(t).latitude for t in countries_df['pays']]
countries_df['lon']=[geolocator.geocode(t).longitude for t in countries_df['pays']]


In [ ]:
traduction['pays']=[countries_dict[t] if t!='nc' else 'nc' for t in traduction['Traduit de']]
traduction['lat']=[countries_df[['lat']].loc[(countries_df.langue==l)&(countries_df.pays==p)].values[0][0] if l!='nc' and p!='nc' else "-" for l,p in zip(traduction['Traduit de'],traduction['pays'])]
traduction['lon']=[countries_df[['lon']].loc[(countries_df.langue==l)&(countries_df.pays==p)].values[0][0] if l!='nc' and p!='nc' else "-" for l,p in zip(traduction['Traduit de'],traduction['pays'])]


In [ ]:
traduction

In [ ]:
traduction_gp = traduction[["ISBN","pays","lat","lon"]].loc[(traduction["Traduit de"]!='nc')&(traduction["Traduit de"]!='VF')&(traduction["TYPES"]=='Romans étrangers')].groupby(["pays","lat","lon"]).count().sort_values('ISBN',ascending=False).reset_index()
traduction_gp.pays = traduction_gp.pays.apply(lambda x : x.capitalize())
traduction_gp

Grap map langue traduite

In [ ]:
# import the library
import folium

# Make an empty map
m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

# Show the map
#m

def color(elev):
    if elev in range(0,10):
        col = 'blue'
    elif elev in range(10,16):
        col = 'darkblue'
    elif elev in range(16,20):
        col = 'pink'
    else:
        col='purple'
    return col


# add marker one by one on the map
for lat,lon,name,elev in zip(traduction_gp['lat'],traduction_gp['lon'],traduction_gp['pays'],traduction_gp['ISBN']):
    folium.Marker(
      location=[lat,lon],
      popup=
        """<i>Pays: </i> <br> <b>{}</b> <br>
           <i>Nb de livres traduits: </i><b><br>{}</b><br>""".format(name,elev),
      icon= folium.Icon(color=color(elev),
      icon_color='white',icon = 'book'),
      tooltip = "Plus d'infos"
   ).add_to(m)

# Show the map again
m


#### Analyse titre ouvrage
Je décide de découper le texte en mots puis de les passer en minuscule.

In [ ]:
rl22.head(3)

In [ ]:
# Create list of word tokens

token_list = []
for token in rl22['RESUME']:
    for doc in nlp(token):
        token_list.append(doc.text)

print(token_list)

In [ ]:
#Cleaning Text Data: Removing Stopwords

#Stop words
#importing stop words from English language.
#import spacy
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS

#Printing the total number of stop words:
print('Number of stop words: %d' % len(spacy_stopwords))

#Printing first ten stop words:
print('First twenty stop words: %s' % list(spacy_stopwords)[:20])

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

#Implementation of stop words:
filtered_sent=[]

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

# filtering stop words
for word in doc:
    if word.is_stop==False:
        filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

In [ ]:
import nltk

tokenizer = nltk.RegexpTokenizer(r'\w+')

df_titre = rl22[['ISBN','TITRE']]
df_titre['corpus_split'] = df_titre['TITRE'].map(lambda x: tokenizer.tokenize(x.lower()) if type(x)==str else str(''))



**Objectif** :
étudier dans un premier temps le vocabulaire utilisé pour chaque titre. Pour une première intuition, il est judicieux d'observer le nombre de mots utilisés.

In [ ]:
df_titre['freq'] = df_titre['corpus_split'].map(lambda x: nltk.FreqDist(x))
df_titre['len_corpus'] = df_titre['corpus_split'].map(lambda x: len(x))
df_titre['len_corpus_unique'] = df_titre['corpus_split'].map(lambda x: len(set(x)))
df_titre.head(10)

In [ ]:
df_titre[['len_corpus','len_corpus_unique']].describe()

In [ ]:
titre_rep = df_titre[['ISBN',"len_corpus_unique"]].groupby(["len_corpus_unique"]).count()#.sort_values('ISBN',ascending=False).reset_index().pivot_table(values='ISBN', index=['GENRE'],columns=['TYPES'], aggfunc=np.sum).reset_index()
titre_rep

**Observations :**
- Le nombre moyen de mots pour un titre est de 4. -> 118 ouvrages
- 119 ouvrages n'ont que 2 mots dans leur titre.
- Le titre le plus long compte 19 mots uniques.
- 48 titres ne comptent qu'1 mot.
- Un plus des 2/3 des ouvarges ont jusqu'à 4 mots dans leur titre.
- 82% des ouvrages ont jusqu'à 5 mots.

In [ ]:
round(((48+119+95+118+81)/565)*100,0)

In [ ]:
fig = px.histogram(df_titre, x="len_corpus_unique",
                   title='Répartition par nombre de mots dans le titre',
                   #labels={'ISBN':'NB OUVRAGES'},
                   histnorm='')


fig.show()

In [ ]:
#table = pd.pivot_table(df_titre, index=['len_corpus_unique'], aggfunc=np.sum)
# Affichage des fréquences
%matplotlib inline

plt.style.use('seaborn-whitegrid')

plt.figure(figsize=(10,6))
table = df_titre.sort_values(by='len_corpus', ascending=False)

table.len_corpus.plot(kind='bar', color="#0b3954", title='Top artiste par nombre de mots')
table.len_corpus_unique.plot(kind='bar', color="#ff6663")

# plot. L'argument loc spécifie le placement de la légende
plt.legend(loc='upper right');

In [ ]:
rl22["TITRE"][46]

In [ ]:
#rl22.to_excel("recap_page_20220825.xlsx")